## Application - Forecasting quarterly US inflation

In [ ]:
# Empty the workspace
rm(list = ls())

# Set Folder-Path
path <- "~/LocalPredictability" # Change to project folder

In [ ]:
# Packages
library("tidyverse")
library("purrr")
library("magrittr")
library("glue")
library("rlang")
library("readr")
library("doParallel")
library("pbapply")
library("ggplot2")
library("cowplot")
library("ggridges")
library("glmnet")
library("ranger")
library("lightgbm")
library("xgboost")
# install.packages(paste0(path, "/eDMA_1.5-3.tar.gz"), repos = NULL, type = "source")
library("eDMA") # Local Version
library("hdflex")
library("forecast")
library("kableExtra")

In [ ]:
# Load Custom Functions
source(glue("{path}/Code/_helpers.R"), local = TRUE)

# Convert Jupyter Notebook to R script
convert_ipynb_to_r(glue("{path}/Code/inflation_main_script.ipynb"))

---

## 1) Forecasting Accuracy

In [ ]:
######### --------------------------------------------
### Predict Quarterly Inflation -- main results ###
# Setting
window_size <-  15 * 4
setting <- glue("inflation_{window_size}")

# Set Global Parameter
cores <- 4
eval_start <- "1991-04-01"
eval_end <- "2021-12-31"

# Set Target-Variable-Names
target_names <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL")

# Set up Parallel-Backend
cl <- parallel::makeCluster(cores)
parallel::clusterExport(cl = cl, varlist = c("setting",
                                             "target_names",
                                             "path",
                                             "window_size",
                                             "eval_start",
                                             "eval_end"),
                        envir = environment())
parallel::clusterEvalQ(cl, {
  library("tidyverse")
  library("magrittr")
  library("purrr")
  library("glue")
  library("rlang")
  library("readr")
  library("doParallel")
  library("glmnet")
  library("xgboost")
  library("lightgbm")
  library("ranger")
  library("eDMA")
  library("hdflex")
})
pbo <- pboptions(type = "timer")

######### --------------------------------------------
### Parallel Loop over Target-Variables ###
res <- pblapply(cl = cl, X = seq_along(target_names), FUN = function(p) {

  ### Load Custom Functions
  source(glue("{path}/Code/_fmodels.R"), local = TRUE)
  source(glue("{path}/Code/_helpers.R"), local = TRUE)

  ######### --------------------------------------------
  ### Build Dataset
  # Response-Name
  y_target <- glue("{target_names[p]}_h1")
  y_signal <- target_names[p]
  not_target <- setdiff(glue("{target_names}_h1"), y_target)

  # Load Y-Data
  y_data <- read_rds(glue("{path}/Data/ES2/P_Signals/y_data.rds")) %>%
    select(-!!not_target)

  # Load Y-Lags
  y_lags <- read_rds(glue("{path}/Data/ES2/P_Signals/y_lags.rds")) %>%
    select(-starts_with(!!not_target))

  # Load P-Signals
  x_data <- read_rds(glue("{path}/Data/ES2/P_Signals/x_data.rds")) %>%
    select(-!!y_signal)

  # Load F-Signals (I)
  cm_signals <- read_rds(glue("{path}/Data/ES2/F_Signals/CM_Forecasts_{y_target}.rds"))

  # Load F-Signals (II)
  refi_signals <- read_rds(glue("{path}/Data/ES2/F_Signals/Forecasts_{y_signal}.rds")) %>%
    select(Date,
           SBAR, UCSV,
           GPR_FAC5, TVD_FAC5, VBDVS_FAC5,
           SSVS_FAC60, VBDVS_FAC60,
           VBDVS_X)

  # Combine, lag and prepare data
  data <- y_data %>%
    dplyr::left_join(y_lags, by = "Date") %>%
    dplyr::left_join(x_data, by = "Date") %>%
    dplyr::mutate(across(-c(1, 2, 3, 4), dplyr::lag)) %>%
    dplyr::left_join(cm_signals, by = "Date") %>%
    dplyr::left_join(refi_signals, by = "Date") %>%
    dplyr::slice(-c(1:3)) %>%
    tibble::column_to_rownames("Date")

  # Assign Response-Variable
  y <- data %>%
    dplyr::select(all_of(y_target)) %>%
    as.matrix()

  # Assign P-Signals
  X <- data %>%
    dplyr::select(all_of(c(colnames(y_lags)[-1], colnames(x_data)[-1]))) %>%
    as.matrix()

  # Assign F-Signals
  Ext_F <- data %>%
    dplyr::select(all_of(c(colnames(cm_signals)[-1], colnames(refi_signals)[-1]))) %>%
    as.matrix()

  # Dates and Observations
  tdates <- rownames(y)
  tlength <- length(tdates)

  # Remove
  rm(list = c("data", "y_data",  "not_target",
              "y_lags", "x_data", "cm_signals", "refi_signals"))

  ######### --------------------------------------------
  ### Build Result Matrices
  # Benchmark Methods
  benchmark_names <- c("AR2", "STAR2",
                       "DART", "ReLa", "TRF",
                       "XGB", "TCSR",
                       #---#
                       "STSC", "STSC_EXF",
                       "STSCS10", "STSCSFLEX",
                       "PC5DMA", "PC10DMA", "PC15DMA")

  # Result-Object: Point Forecasts
  preds <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                  dimnames = list(tdates, benchmark_names))

  # Result-Object: Variances
  vari <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                 dimnames = list(tdates, benchmark_names))

  # Result-Object: Squared Errors
  se <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
               dimnames = list(tdates, benchmark_names))

  # Result-Object: Continuous Ranked Probability Score
  crps_score <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                       dimnames = list(tdates, benchmark_names))

  ######### --------------------------------------------
  ### Part 1: Benchmark ###
  ### Rolling AR(2)-Model
  # Time Sequence
  t_seq <- seq(window_size, nrow(y) - 1)

  # Rolling Loop
  for (t in t_seq) {

    ### Split Data ###
    # Train Data
    x_train <- X[(t - window_size + 1):t, 1:2, drop = FALSE]
    y_train <- y[(t - window_size + 1):t, , drop = FALSE]

    # Predict Data
    x_pred <- X[t + 1, 1:2, drop = FALSE]
    y_pred <- y[t + 1, ]

    # Fit and Predict
    pred_ar2 <- ar_model(x_train,
                         y_train,
                         x_pred,
                         2)
    # Save
    preds[t + 1, "AR2"] <- pred_ar2
    vari[t + 1, "AR2"] <- var(y_train)
    se[t + 1, "AR2"] <- (y_pred - pred_ar2) ** 2
  }

  ### Density Transformation (ST-AR2) ###
  # Apply Function
  star2_results <- hdflex::tvc(y,
                               NULL,
                               preds[, "AR2", drop = FALSE],
                               5 * 4,
                               1.00,
                               0.98,
                               FALSE)

  # Assign STSC-Results
  preds[, "STAR2"] <- star2_results$Forecasts$Point_Forecasts
  vari[, "STAR2"] <- star2_results$Forecasts$Variance_Forecasts
  se[, "STAR2"] <- (y - preds[, "STAR2"]) ** 2
  crps_score[, "STAR2"] <- crps(y,
                                preds[, "STAR2"],
                                sqrt(vari[, "STAR2"]),
                                NULL, "normal")$crps

  # Remove & Clean Memory
  rm(list = c("x_train", "y_train", "x_pred", "y_pred",
              "pred_ar2", "star2_results", "t_seq"))

  ######### --------------------------------------------
  ### Part 2: Forecasting Models with Rolling Window ###
  # Combine, remove NA-rows and subset response
  S_sub <- na.omit(cbind(X, Ext_F))
  y_sub <- y[rownames(S_sub), , drop = FALSE]

  # Get Start Point for Training
  train_start <- sum(rownames(S_sub) < eval_start)
  if (train_start < window_size) {
    rlang::abort("Dataset is too small for the given window size.")
  }

  # First Complete Observation
  first_complete <- which(complete.cases(cbind(X, Ext_F)))[1]
  adj <- first_complete - 1

  # Time Sequence
  t_seq <- seq(train_start,  nrow(S_sub) - 1)

  # Loop with Rolling Window
  for (t in t_seq) {

    ### Split Data ###
    # Train Data
    s_train <- S_sub[(t - window_size + 1):t, , drop = FALSE]
    y_train <- y_sub[(t - window_size + 1):t, , drop = FALSE]

    # Predict Data
    s_pred <- S_sub[t + 1, , drop = FALSE]
    y_pred <- y_sub[t + 1, ]

    ####### Model 1: Relaxed-Lasso #######
    # Parameter
    rela_folds <- 5

    # Fit and Predict
    pred_relax <- relasso_model(s_train,
                                y_train,
                                s_pred,
                                rela_folds,
                                t)

    ####### Model 2: XGBoost #######
    # Parameter
    xgb_folds  <- 5
    xgb_ntrees <- 500
    xgb_lr     <- 0.1
    xgb_target <- NULL
    xgb_cores  <- 1

    # Predict
    pred_xgb <- xgb_model(s_train,
                          y_train,
                          s_pred,
                          xgb_folds,
                          xgb_ntrees,
                          xgb_lr,
                          xgb_target,
                          xgb_cores,
                          t)

    ####### Model 3: LightGBM #######
    # Parameter
    dart_folds  <- 5
    dart_ntrees <- 500
    dart_lr     <- 0.1
    dart_dr     <- 0.1
    dart_target <- NULL
    dart_cores  <- 1

    # Fit and Predict
    pred_dart <- dart_model(s_train,
                            y_train,
                            s_pred,
                            dart_folds,
                            dart_ntrees,
                            dart_lr,
                            dart_dr,
                            dart_target,
                            dart_cores,
                            t)

    ####### Model 4: (Targeted) RandomForests #######
    # Parameter
    trf_ntarget   <- 50
    trf_ntrees    <- 500
    trf_max_depth <- 3
    trf_cores     <- 1

    # Fit and Predict
    pred_trf <- trf_model(s_train,
                          y_train,
                          s_pred,
                          trf_ntarget,
                          trf_ntrees,
                          trf_max_depth,
                          trf_cores,
                          t)

    ####### Model 5: (Targeted) Complete Subset #######
    # Parameter
    csr_n_target <- 20
    csr_n_subset <- 10
    csr_ubound   <- 10000
    csr_sampling <- TRUE

    # Fit and Predict
    pred_csr <- csr_model(s_train,
                          y_train,
                          s_pred,
                          csr_n_target,
                          csr_n_subset,
                          csr_ubound,
                          csr_sampling)

    ####### Save Point Forecasts #######
    preds[t + 1 + adj, "ReLa"] <- pred_relax
    preds[t + 1 + adj, "XGB"]  <- pred_xgb
    preds[t + 1 + adj, "DART"] <- pred_dart
    preds[t + 1 + adj, "TRF"]  <- pred_trf
    preds[t + 1 + adj, "TCSR"] <- pred_csr

    ###### Save Variances #######
    vari[t + 1 + adj, "ReLa"] <- var(y_train)
    vari[t + 1 + adj, "XGB"]  <- var(y_train)
    vari[t + 1 + adj, "DART"] <- var(y_train)
    vari[t + 1 + adj, "TRF"]  <- var(y_train)
    vari[t + 1 + adj, "TCSR"] <- var(y_train)

    ####### Save Squared Errors #######
    se[t + 1 + adj, "ReLa"] <- (y_pred - pred_relax) ** 2
    se[t + 1 + adj, "XGB"]  <- (y_pred - pred_xgb) ** 2
    se[t + 1 + adj, "DART"] <- (y_pred - pred_dart) ** 2
    se[t + 1 + adj, "TRF"]  <- (y_pred - pred_trf) ** 2
    se[t + 1 + adj, "TCSR"] <- (y_pred - pred_csr) ** 2
  }

  # Remove & Clean Memory
  rm(list = c("s_train", "y_train", "s_pred", "y_pred", "S_sub", "y_sub",
              "pred_relax", "pred_xgb", "pred_dart",
              "pred_trf", "pred_csr",
              "t_seq"))
  invisible(gc())

  ######### --------------------------------------------
  ### Part 2: STSC and Variants ###
  ####### STSC-Model #######
  # Set TV-C-Parameter
  init <- 5 * 4
  lambda_grid <- c(0.90, 0.95, 1.00)
  kappa_grid <- c(0.94, 0.96, 0.98)
  bias <- TRUE

  # Set DSC-Parameter
  gamma_grid <- c(0.40, 0.50, 0.60, 0.70, 0.80, 0.90,
                  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.00)
  n_tvc <- (ncol(X) + ncol(Ext_F)) * length(lambda_grid) * length(kappa_grid)
  psi_grid <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc / 4)))
  delta <- 0.95
  burn_in <- first_complete + init / 2
  burn_in_dsc <- 1
  metric <- 5
  equal_weight <- TRUE
  incl <- NULL
  parallel <- FALSE
  n_threads <- NULL

  # Apply Function
  stsc_results <- hdflex::stsc(y,
                               X,
                               Ext_F,
                               init,
                               lambda_grid,
                               kappa_grid,
                               bias,
                               gamma_grid,
                               psi_grid,
                               delta,
                               burn_in,
                               burn_in_dsc,
                               metric,
                               equal_weight,
                               incl,
                               parallel,
                               n_threads,
                               NULL)

  # Assign STSC-Results
  pred_stsc <- stsc_results$Forecasts$Point_Forecasts
  var_stsc <- stsc_results$Forecasts$Variance_Forecasts
  chosen_gamma <- stsc_results$Tuning_Parameters$Gamma
  chosen_psi <- stsc_results$Tuning_Parameters$Psi
  chosen_signals <- stsc_results$Tuning_Parameters$Signals
  chosen_lambda <- stsc_results$Tuning_Parameters$Lambda
  chosen_kappa <- stsc_results$Tuning_Parameters$Kappa

  # Add dates
  names(pred_stsc) <- tdates
  names(var_stsc) <- tdates
  names(chosen_gamma) <- tdates
  names(chosen_psi) <- tdates
  rownames(chosen_signals) <- tdates
  rownames(chosen_lambda) <- tdates
  rownames(chosen_kappa) <- tdates

  # Remove & Clean Memory
  rm(list = c("stsc_results"))

  ####### STSC - Excluding-F-Signals #######
  n_tvc_exf <- (ncol(X) + 0) * length(lambda_grid) * length(kappa_grid)
  psi_grid_exf <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc_exf / 4)))

  # Apply Function
  stsc_exf_results <- hdflex::stsc(y,
                                   X,
                                   NULL,
                                   init,
                                   lambda_grid,
                                   kappa_grid,
                                   bias,
                                   gamma_grid,
                                   psi_grid_exf,
                                   delta,
                                   burn_in,
                                   burn_in_dsc,
                                   metric,
                                   equal_weight,
                                   incl,
                                   parallel,
                                   n_threads,
                                   NULL)

  # Assign Results
  pred_stsc_exf <- stsc_exf_results$Forecasts$Point_Forecasts
  var_stsc_exf  <- stsc_exf_results$Forecasts$Variance_Forecasts

  # Remove & Clean Memory
  rm(list = c("stsc_exf_results"))

  ####### STSCSx - Selecting 10 Forecasts with LASSO #######
  # Set TV-C-Parameter
  init <- 5 * 4
  lambda_grid <- c(0.90, 0.95, 1.00)
  kappa_grid <- c(0.94, 0.96, 0.98)
  bias <- TRUE
  n_cores <- 1

  # Set Lasso-Target-Parameter
  stscsx_target <- 10

  # Apply STSC-S-X Function
  stscsx_results <- stscsx(y,
                           X,
                           Ext_F,
                           lambda_grid,
                           kappa_grid,
                           init,
                           bias,
                           n_cores,
                           window_size,
                           stscsx_target)

  # Assign Results
  pred_stscsx <- stscsx_results[, 1]
  var_stscsx  <- stscsx_results[, 2]

  # Remove & Clean Memory
  rm(list = c("stscsx_results"))

  ####### STSC - Using LASSO to dynamically select forecasts #######
  # Set TV-C-Parameter
  init <- 5 * 4
  lambda_grid <- c(0.90, 0.95, 1.00)
  kappa_grid <- c(0.94, 0.96, 0.98)
  bias <- TRUE
  n_cores <- 1

  # Set Lasso-CV-Parameter
  stscsflex_folds <- 5

  # Apply STSC-S-FLEX Function
  stscsflex_results <- stscsflex(y,
                                 X,
                                 Ext_F,
                                 lambda_grid,
                                 kappa_grid,
                                 init,
                                 bias,
                                 n_cores,
                                 window_size,
                                 stscsflex_folds)

  # Assign Results
  pred_stscsflex <- stscsflex_results[, 1]
  var_stscsflex  <- stscsflex_results[, 2]

  # Remove & Clean Memory
  rm(list = c("stscsflex_results"))

  ####### (Principal Component) Dynamic Model Averaging #######
  # Set Parameter
  pcdma_win    <- window_size
  pcdma_comp   <- 3
  pcdma_alpha  <- 0.99
  pcdma_lambda <- c(0.90, 0.95, 1.00)
  pcdma_kappa  <- 0.98
  pcdma_cores  <- 1
  pcdma_excl   <- c(1, 2)

  # Apply PCA-DMA
  pc5dma_results <- pcdma(y,
                          X,
                          Ext_F,
                          pcdma_win,
                          pcdma_comp,
                          pcdma_alpha,
                          pcdma_lambda,
                          pcdma_kappa,
                          pcdma_cores,
                          pcdma_excl)

  # Assign Results
  pred_pc5dma <- pc5dma_results[, 1]
  var_pc5dma  <- pc5dma_results[, 3]
  crps_pc5dma <- pc5dma_results[, 4]

  # Remove & Clean Memory
  rm(list = c("pc5dma_results"))

  # Set Parameter
  pcdma_comp <- 8

  # Apply PCA-DMA
  pc10dma_results <- pcdma(y,
                           X,
                           Ext_F,
                           pcdma_win,
                           pcdma_comp,
                           pcdma_alpha,
                           pcdma_lambda,
                           pcdma_kappa,
                           pcdma_cores,
                           pcdma_excl)

  # Assign Results
  pred_pc10dma <- pc10dma_results[, 1]
  var_pc10dma  <- pc10dma_results[, 3]
  crps_pc10dma <- pc10dma_results[, 4]

  # Remove & Clean Memory
  rm(list = c("pc10dma_results"))

  # Set Parameter
  pcdma_comp <- 13

  # Apply PCA-DMA
  pc15dma_results <- pcdma(y,
                           X,
                           Ext_F,
                           pcdma_win,
                           pcdma_comp,
                           pcdma_alpha,
                           pcdma_lambda,
                           pcdma_kappa,
                           pcdma_cores,
                           pcdma_excl)

  # Assign Results
  pred_pc15dma <- pc15dma_results[, 1]
  var_pc15dma  <- pc15dma_results[, 3]
  crps_pc15dma <- pc15dma_results[, 4]

  # Remove & Clean Memory
  rm(list = c("pc15dma_results"))

  ######### --------------------------------------------
  ####### Save Point Forecasts #######
  preds[, "STSC"] <- pred_stsc
  preds[, "STSC_EXF"] <- pred_stsc_exf
  preds[, "STSCS10"] <- pred_stscsx
  preds[, "STSCSFLEX"] <- pred_stscsflex
  preds[, "PC5DMA"] <- pred_pc5dma
  preds[, "PC10DMA"] <- pred_pc10dma
  preds[, "PC15DMA"] <- pred_pc15dma

  ####### Save Variances #######
  vari[, "STSC"] <- var_stsc
  vari[, "STSC_EXF"] <- var_stsc_exf
  vari[, "STSCS10"] <- var_stscsx
  vari[, "STSCSFLEX"] <- var_stscsflex
  vari[, "PC5DMA"] <- var_pc5dma
  vari[, "PC10DMA"] <- var_pc10dma
  vari[, "PC15DMA"] <- var_pc15dma

  ####### Save Squared Errors #######
  se[, "STSC"] <- (y - pred_stsc) ** 2
  se[, "STSC_EXF"] <- (y - pred_stsc_exf) ** 2
  se[, "STSCS10"] <- (y - pred_stscsx) ** 2
  se[, "STSCSFLEX"] <- (y - pred_stscsflex) ** 2
  se[, "PC5DMA"] <- (y - pred_pc5dma) ** 2
  se[, "PC10DMA"] <- (y - pred_pc10dma) ** 2
  se[, "PC15DMA"] <- (y - pred_pc15dma) ** 2

  ####### Save Continuous Ranked Probability Score #######
  crps_score[, "STSC"] <- crps(y, pred_stsc, sqrt(var_stsc), NULL, "normal")$crps # nolint
  crps_score[, "STSC_EXF"] <- crps(y, pred_stsc_exf, sqrt(var_stsc_exf), NULL, "normal")$crps # nolint
  crps_score[, "STSCS10"] <- crps(y, pred_stscsx, sqrt(var_stscsx), NULL, "normal")$crps # nolint
  crps_score[, "STSCSFLEX"] <- crps(y, pred_stscsflex, sqrt(var_stscsflex), NULL, "normal")$crps # nolint
  crps_score[, "PC5DMA"]  <- crps_pc5dma
  crps_score[, "PC10DMA"] <- crps_pc10dma
  crps_score[, "PC15DMA"] <- crps_pc15dma

  # Remove & Clean Memory
  rm(list = c("pred_stsc", "pred_stsc_exf",
              "pred_stscsx", "pred_stscsflex",
              "pred_pc5dma", "pred_pc10dma", "pred_pc15dma",
              "var_stsc", "var_stsc_exf",
              "var_stscsflex", "var_stscsx",
              "var_pc5dma", "var_pc10dma", "var_pc15dma",
              "crps_pc5dma", "crps_pc10dma", "crps_pc15dma"))
  invisible(gc())

  ######### --------------------------------------------
  ### Part 3: Evaluation ###
  ####### OOS-Period #######
  # Define Evaluation Period (OOS-Period)
  oos_idx <- which(tdates >= eval_start & tdates <= eval_end)

  # OOS-Data
  oos_y <- y[oos_idx, ]
  oos_preds <- preds[oos_idx, ]
  oos_vari <- vari[oos_idx, ]
  oos_se <- se[oos_idx, ]
  oos_crps <- crps_score[oos_idx, ]
  oos_chosen_gamma <- chosen_gamma[oos_idx]
  oos_chosen_psi <- chosen_psi[oos_idx]
  oos_chosen_signals <- chosen_signals[oos_idx, ]
  oos_chosen_lambda <- chosen_lambda[oos_idx, ]
  oos_chosen_kappa <- chosen_kappa[oos_idx, ]

  # Check for Missing Values
  for (obj in list(oos_y, oos_preds, oos_vari, oos_se)) {
    if (any(is.na(obj))) {
      rlang::abort("Error: Missing Values in one of the OOS objects")
    }
  }

  ####### Mean-Squared-Errors #######
  mse <- colMeans(oos_se)
  mse <- matrix(mse, nrow = 1, ncol = length(benchmark_names),
                dimnames = list(y_signal, benchmark_names))

  ####### Mean-Continuous Ranked Probability Score #######
  mcrps <- colMeans(oos_crps)
  mcrps <- matrix(mcrps, nrow = 1, ncol = length(benchmark_names),
                  dimnames = list(y_signal, benchmark_names))

  ######### --------------------------------------------
  # Save Results
  write_rds(oos_y, glue("{path}/Results/ES2/{setting}_y_{y_signal}.rds"))
  write_rds(oos_preds, glue("{path}/Results/ES2/{setting}_point_forecasts_{y_signal}.rds"))
  write_rds(oos_vari, glue("{path}/Results/ES2/{setting}_variances_{y_signal}.rds"))
  write_rds(oos_se, glue("{path}/Results/ES2/{setting}_squared_errors_{y_signal}.rds"))
  write_rds(oos_crps, glue("{path}/Results/ES2/{setting}_crps_scores_{y_signal}.rds"))
  write_rds(list(mse = mse, crps = mcrps), glue("{path}/Results/ES2/{setting}_res_{y_signal}.rds"))

  # Save STSC-Parameter
  write_rds(list(
    gamma = oos_chosen_gamma,
    psi = oos_chosen_psi,
    signals = oos_chosen_signals,
    lambda = oos_chosen_lambda,
    kappa = oos_chosen_kappa
  ), glue("{path}/Results/ES2/{setting}_stsc_params_{y_signal}.rds"))

  # Return Results
  return(list(mse = mse, crps = mcrps))
  ######### --------------------------------------------
})

# Stop Cluster
parallel::stopCluster(cl)

# Mean-Squared-Errors
res_mse <- do.call("rbind", purrr::map(res, 1))
round(res_mse / res_mse[, "STSC"], 2)

# Continuous Ranked Probability Score
res_crps <- do.call("rbind", purrr::map(res, 2))
round(res_crps / res_crps[, "STSC"], 2)
######### --------------------------------------------

## 2.) Statistical-Tests

In [ ]:
######### --------------------------------------------
### Diebold-Mariano Test - MSE and CRPS ###
# Setting
setting <- "inflation_60"

# Set Target-Variable-Names
target_names <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL")

# Build Result-Object
results <- tibble(
  Target = character(),
  Forecast = character(),
  Statistic = numeric(),
  P.Value = numeric(),
  Alternative = character(),
  TestType = character()
)

# Loop over target variable
for (p in 1:4) {

  # Load Data
  y_signal <- target_names[p]
  y <- read_rds(glue("{path}/Results/ES2/{setting}_y_{y_signal}.rds"))
  preds <- read_rds(glue("{path}/Results/ES2/{setting}_point_forecasts_{y_signal}.rds"))
  crps_scores <- read_rds(glue("{path}/Results/ES2/{setting}_crps_scores_{y_signal}.rds"))

  # Loop over each forecast method for MSE
  for (forecast_method in colnames(preds)) {
    if (forecast_method != "AR2" && forecast_method != "STAR2") {

      # Forecast errors
      errors_benchmark <- y - preds[, "AR2", drop = FALSE]
      errors_forecast <- y - preds[, forecast_method, drop = FALSE]

      # Perform DM-Test
      dm_test_result <- forecast::dm.test(errors_benchmark,
                                          errors_forecast,
                                          alternative = "greater",
                                          h = 1,
                                          power = 2)

      # Append the result
      results <- results %>%
        add_row(
          Target = y_signal,
          Forecast = forecast_method,
          Statistic = round(dm_test_result$statistic, 4),
          P.Value = round(dm_test_result$p.value, 4),
          Alternative = dm_test_result$alternative,
          TestType = "MSE"
        )
    }
  }

  # Loop over each forecast method for CRPS
  for (forecast_method in colnames(crps_scores)) {
    if (forecast_method != "AR2" && forecast_method != "STAR2") {

      # Forecast Errors
      errors_benchmark <- crps_scores[, "STAR2", drop = FALSE]
      errors_forecast <- crps_scores[, forecast_method, drop = FALSE]

      # Check for missing values (Non-Density-Methods)
      if (!any(is.na(errors_forecast))) {

        # Perform DM-Test
        dm_test_result <- forecast::dm.test(errors_benchmark,
                                            errors_forecast,
                                            alternative = "greater",
                                            h = 1,
                                            power = 1)

        # Append the result
        results <- results %>%
          add_row(
            Target = y_signal,
            Forecast = forecast_method,
            Statistic = round(dm_test_result$statistic, 4),
            P.Value = round(dm_test_result$p.value, 4),
            Alternative = dm_test_result$alternative,
            TestType = "CRPS"
          )
      }
    }
  }
}

# Show results
results %>%
  filter(P.Value < 0.10)
######### --------------------------------------------

In [ ]:
######### --------------------------------------------
### Probability Integral Transformation (PIT)
# Setting
setting <- "inflation_60"

# Target Variables
target_names  <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL")

# Build Matrix
cdf_t <- matrix(NA, nrow = 123, ncol = 4,
                dimnames = list(NULL, c("GDP Deflator",
                                        "PCE Deflator",
                                        "Total CPI",
                                        "Core CPI")))

# Loop over target variables
for (p in 1:4) {

  # Load Y
  y_signal <- target_names[p]
  y <- read_rds(glue("{path}/Results/ES2/{setting}_y_{y_signal}.rds"))

  # Load Predictive Density
  pred_stsc <- read_rds(glue("{path}/Results/ES2/{setting}_point_forecasts_{y_signal}.rds"))[, "STSC"]
  var_stsc <- read_rds(glue("{path}/Results/ES2/{setting}_variances_{y_signal}.rds"))[, "STSC"]
  data <- cbind(y, cbind(pred_stsc, var_stsc))

  # Transform to Uniform (Cumulated Predictive Density Function evaluated at realisation)
  cdf_t[, p] = apply(data, 1, FUN = function(x) pnorm(q = x[1], mean = x[2], sd = sqrt(x[3])))

  # Kolmogorov-Smirnov-Test
  ks_test <- stats::ks.test(cdf_t[, p], "punif", 0, 1)
  print(paste("Kolmogorov-Smirnov Test p-value:", round(ks_test$p.value, 4)))
}

# Save Uniform Vectors
write_rds(cdf_t, glue("{path}/Results/ES2/pit_cdf_t.rds"))
######### --------------------------------------------

## 3) Alternative Tuning Parameter

In [ ]:
######### --------------------------------------------
### Predict Quarterly Inflation -- Robustness ###
# Setting
setting <- "robustness"

# Set Global Parameter
cores <- 4
window_size <- 15 * 4
eval_start <- "1991-04-01"
eval_end <- "2021-12-31"

# Set Target-Variable-Names
target_names <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL")

# Set up Parallel-Backend
cl <- parallel::makeCluster(cores)
parallel::clusterExport(cl = cl, varlist = c("setting",
                                             "target_names",
                                             "path",
                                             "window_size",
                                             "eval_start",
                                             "eval_end"),
                        envir = environment())
parallel::clusterEvalQ(cl, {
  library("tidyverse")
  library("magrittr")
  library("purrr")
  library("glue")
  library("rlang")
  library("readr")
  library("doParallel")
  library("hdflex")
})
pbo <- pboptions(type = "timer")

######### --------------------------------------------
### Parallel Loop
res <- pblapply(cl = cl, X = seq_along(target_names), FUN = function(p) {

  ### Load Custom Functions
  source(glue("{path}/Code/_fmodels.R"), local = TRUE)
  source(glue("{path}/Code/_helpers.R"), local = TRUE)

  ######### --------------------------------------------
  ### Build Dataset
  # Response-Name
  y_target <- glue("{target_names[p]}_h1")
  y_signal <- target_names[p]
  not_target <- setdiff(glue("{target_names}_h1"), y_target)

  # Load Y-Data
  y_data <- read_rds(glue("{path}/Data/ES2/P_Signals/y_data.rds")) %>%
    select(-!!not_target)

  # Load Y-Lags
  y_lags <- read_rds(glue("{path}/Data/ES2/P_Signals/y_lags.rds")) %>%
    select(-starts_with(!!not_target))

  # Load P-Signals
  x_data <- read_rds(glue("{path}/Data/ES2/P_Signals/x_data.rds")) %>%
    select(-!!y_signal)

  # Load F-Signals (I)
  cm_signals <- read_rds(glue("{path}/Data/ES2/F_Signals/CM_Forecasts_{y_target}.rds"))

  # Load F-Signals (II)
  refi_signals <- read_rds(glue("{path}/Data/ES2/F_Signals/Forecasts_{y_signal}.rds")) %>%
    select(Date,
           SBAR, UCSV,
           GPR_FAC5, TVD_FAC5, VBDVS_FAC5,
           SSVS_FAC60, VBDVS_FAC60,
           VBDVS_X)

  # Combine, lag and prepare data
  data <- y_data %>%
    dplyr::left_join(y_lags, by = "Date") %>%
    dplyr::left_join(x_data, by = "Date") %>%
    dplyr::mutate(across(-c(1, 2, 3, 4), dplyr::lag)) %>%
    dplyr::left_join(cm_signals, by = "Date") %>%
    dplyr::left_join(refi_signals, by = "Date") %>%
    dplyr::slice(-c(1:3)) %>%
    tibble::column_to_rownames("Date")

  # Assign Response-Variable
  y <- data %>%
    dplyr::select(all_of(y_target)) %>%
    as.matrix()

  # Assign P-Signals
  X <- data %>%
    dplyr::select(all_of(c(colnames(y_lags)[-1], colnames(x_data)[-1]))) %>%
    as.matrix()

  # Assign F-Signals
  Ext_F <- data %>%
    dplyr::select(all_of(c(colnames(cm_signals)[-1], colnames(refi_signals)[-1]))) %>%
    as.matrix()

  # Dates and Observations
  tdates <- rownames(y)
  tlength <- length(tdates)

  # First Non-NA-Row
  first_complete <- which(complete.cases(cbind(X, Ext_F)))[1]

  # Remove
  rm(list = c("data", "y_data",  "not_target",
              "y_lags", "x_data", "cm_signals", "refi_signals"))

  ######### --------------------------------------------
  ### Build Result Matrices
  # Benchmark Methods
  benchmark_names <- c("AR2", "STAR2",
                       #---#
                       "STSC",
                       paste0("STSC", seq(32)),
                       "STSC_NF")

  # Result-Object: Point Forecasts
  preds <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                  dimnames = list(tdates, benchmark_names))

  # Result-Object: Variances
  vari <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                 dimnames = list(tdates, benchmark_names))

  # Result-Object: Squared Errors
  se <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
               dimnames = list(tdates, benchmark_names))

  # Result-Object: Continuous Ranked Probability Score
  crps_score <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                       dimnames = list(tdates, benchmark_names))

  ######### --------------------------------------------
  ### Part 1: Benchmark ###
  ### Rolling AR(2)-Model
  # Time Sequence
  t_seq <- seq(window_size, nrow(y) - 1)

  # Rolling Loop
  for (t in t_seq) {

    ### Split Data ###
    # Train Data
    x_train <- X[(t - window_size + 1):t, 1:2, drop = FALSE]
    y_train <- y[(t - window_size + 1):t, , drop = FALSE]

    # Predict Data
    x_pred <- X[t + 1, 1:2, drop = FALSE]
    y_pred <- y[t + 1, ]

    # Fit and Predict
    pred_ar2 <- ar_model(x_train,
                         y_train,
                         x_pred,
                         2)
    # Save
    preds[t + 1, "AR2"] <- pred_ar2
    vari[t + 1, "AR2"] <- var(y_train)
    se[t + 1, "AR2"] <- (y_pred - pred_ar2) ** 2
  }

  ### Density Transformation (ST-AR2) ###
  # Apply Function
  star2_results <- hdflex::tvc(y,
                               NULL,
                               preds[, "AR2", drop = FALSE],
                               5 * 4,
                               1.00,
                               0.98,
                               FALSE)

  # Assign STSC-Results
  preds[, "STAR2"] <- star2_results$Forecasts$Point_Forecasts
  vari[, "STAR2"] <- star2_results$Forecasts$Variance_Forecasts
  se[, "STAR2"] <- (y - preds[, "STAR2"]) ** 2
  crps_score[, "STAR2"] <- crps(y,
                                preds[, "STAR2"],
                                sqrt(vari[, "STAR2"]),
                                NULL, "normal")$crps

  # Remove & Clean Memory
  rm(list = c("x_train", "y_train", "x_pred", "y_pred",
              "pred_ar2", "star2_results", "t_seq"))

  ######### --------------------------------------------
  ### Part 2: STSC and Variants ###
  ####### STSC-Model - Default #######
  # Set TV-C-Parameter
  init <- 5 * 4
  lambda_grid <- c(0.90, 0.95, 1.00)
  kappa_grid <- c(0.94, 0.96, 0.98)
  bias <- TRUE

  # Set DSC-Parameter
  gamma_grid <- c(0.40, 0.50, 0.60, 0.70, 0.80, 0.90,
                  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.00)
  n_tvc <- (ncol(X) + ncol(Ext_F)) * length(lambda_grid) * length(kappa_grid)
  psi_grid <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc / 4)))
  delta <- 0.95
  burn_in <- first_complete + init / 2
  burn_in_dsc <- 1
  metric <- 5
  equal_weight <- TRUE
  incl <- NULL
  parallel <- FALSE
  n_threads <- NULL

  # Apply Function
  stsc_results <- hdflex::stsc(y,
                               X,
                               Ext_F,
                               init,
                               lambda_grid,
                               kappa_grid,
                               bias,
                               gamma_grid,
                               psi_grid,
                               delta,
                               burn_in,
                               burn_in_dsc,
                               metric,
                               equal_weight,
                               incl,
                               parallel,
                               n_threads,
                               NULL)

  # Assign Results
  preds[, "STSC"] <- stsc_results$Forecasts$Point_Forecasts
  vari[, "STSC"] <- stsc_results$Forecasts$Variance_Forecasts
  se[, "STSC"] <- (y - stsc_results$Forecasts$Point_Forecasts) ** 2
  crps_score[, "STSC"] <- crps(y, stsc_results$Forecasts$Point_Forecasts,
                               sqrt(stsc_results$Forecasts$Variance_Forecasts), NULL, "normal")$crps

  # Remove & Clean Memory
  rm(list = c("stsc_results"))

  ####### Varying Kappa #######
  i <- 1
  for (k in c(0.86, 0.88, 0.90, 0.92, 0.94, 0.96, 0.98, 1.00)) {

    # Print
    cat("Kappa:", k, "\n")

    # Set Parameter
    n_tvc_k <- (ncol(X) + ncol(Ext_F)) * length(lambda_grid) * 1
    psi_grid_k <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc_k / 4)))

    # Apply Function
    stsc_results <- hdflex::stsc(y,
                                 X,
                                 Ext_F,
                                 init,
                                 lambda_grid,
                                 k,
                                 bias,
                                 gamma_grid,
                                 psi_grid_k,
                                 delta,
                                 burn_in,
                                 burn_in_dsc,
                                 metric,
                                 equal_weight,
                                 incl,
                                 parallel,
                                 n_threads,
                                 NULL)

    # Results
    preds[, paste0("STSC", i)] <- stsc_results$Forecasts$Point_Forecasts
    vari[, paste0("STSC", i)] <- stsc_results$Forecasts$Variance_Forecasts
    se[, paste0("STSC", i)] <- (y - stsc_results$Forecasts$Point_Forecasts) ** 2
    crps_score[, paste0("STSC", i)] <- crps(y, stsc_results$Forecasts$Point_Forecasts,
                                            sqrt(stsc_results$Forecasts$Variance_Forecasts), NULL, "normal")$crps

    # Remove & Clean Memory
    rm(list = c("stsc_results"))

    # Update
    i <- i + 1
  }

  ####### Varying Lambda #######
  for (l in c(0.900, 0.925, 0.950, 0.975, 1.000)) {

    # Print
    cat("Lambda:", l, "\n")

    # Set Parameter
    n_tvc_l <- (ncol(X) + ncol(Ext_F)) * 1 * length(kappa_grid)
    psi_grid_l <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc_l / 4)))

    # Apply Function
    stsc_results <- hdflex::stsc(y,
                                 X,
                                 Ext_F,
                                 init,
                                 l,
                                 kappa_grid,
                                 bias,
                                 gamma_grid,
                                 psi_grid_l,
                                 delta,
                                 burn_in,
                                 burn_in_dsc,
                                 metric,
                                 equal_weight,
                                 incl,
                                 parallel,
                                 n_threads,
                                 NULL)

    # Results
    preds[, paste0("STSC", i)] <- stsc_results$Forecasts$Point_Forecasts
    vari[, paste0("STSC", i)] <- stsc_results$Forecasts$Variance_Forecasts
    se[, paste0("STSC", i)] <- (y - stsc_results$Forecasts$Point_Forecasts) ** 2
    crps_score[, paste0("STSC", i)] <- crps(y, stsc_results$Forecasts$Point_Forecasts,
                                            sqrt(stsc_results$Forecasts$Variance_Forecasts), NULL, "normal")$crps

    # Remove & Clean Memory
    rm(list = c("stsc_results"))

    # Update
    i <- i + 1
  }

  ####### Varying Delta #######
  for (d in c(0.900, 0.925, 0.950, 0.975, 1.000)) {

    # Apply Function
    stsc_results <- hdflex::stsc(y,
                                 X,
                                 Ext_F,
                                 init,
                                 lambda_grid,
                                 kappa_grid,
                                 bias,
                                 gamma_grid,
                                 psi_grid,
                                 d,
                                 burn_in,
                                 burn_in_dsc,
                                 metric,
                                 equal_weight,
                                 incl,
                                 parallel,
                                 n_threads,
                                 NULL)

    # Results
    preds[, paste0("STSC", i)] <- stsc_results$Forecasts$Point_Forecasts
    vari[, paste0("STSC", i)] <- stsc_results$Forecasts$Variance_Forecasts
    se[, paste0("STSC", i)] <- (y - stsc_results$Forecasts$Point_Forecasts) ** 2
    crps_score[, paste0("STSC", i)] <- crps(y, stsc_results$Forecasts$Point_Forecasts,
                                            sqrt(stsc_results$Forecasts$Variance_Forecasts), NULL, "normal")$crps

    # Remove & Clean Memory
    rm(list = c("stsc_results"))

    # Update
    i <- i + 1
  }

  ####### Varying Psi #######
  for (p in c(1, 5, 10, 25, 50, 100, 500, 4149)) {

    # Print
    cat("Psi:", p, "\n")

    # Apply Function
    stsc_results <- hdflex::stsc(y,
                                 X,
                                 Ext_F,
                                 init,
                                 lambda_grid,
                                 kappa_grid,
                                 bias,
                                 gamma_grid,
                                 p,
                                 delta,
                                 burn_in,
                                 burn_in_dsc,
                                 metric,
                                 equal_weight,
                                 incl,
                                 parallel,
                                 n_threads,
                                 NULL)

    # Results
    preds[, paste0("STSC", i)] <- stsc_results$Forecasts$Point_Forecasts
    vari[, paste0("STSC", i)] <- stsc_results$Forecasts$Variance_Forecasts
    se[, paste0("STSC", i)] <- (y - stsc_results$Forecasts$Point_Forecasts) ** 2
    crps_score[, paste0("STSC", i)] <- crps(y, stsc_results$Forecasts$Point_Forecasts,
                                            sqrt(stsc_results$Forecasts$Variance_Forecasts), NULL, "normal")$crps

    # Remove & Clean Memory
    rm(list = c("stsc_results"))

    # Update
    i <- i + 1
  }

  ####### Varying Gamma #######
  for (g in c(0.40, 0.60, 0.80, 0.95, 0.99, 1.00)) {

    # Print
    cat("Gamma:", g, "\n")

    # Apply Function
    stsc_results <- hdflex::stsc(y,
                                 X,
                                 Ext_F,
                                 init,
                                 lambda_grid,
                                 kappa_grid,
                                 bias,
                                 g,
                                 psi_grid,
                                 delta,
                                 burn_in,
                                 burn_in_dsc,
                                 metric,
                                 equal_weight,
                                 incl,
                                 parallel,
                                 n_threads,
                                 NULL)

    # Results
    preds[, paste0("STSC", i)] <- stsc_results$Forecasts$Point_Forecasts
    vari[, paste0("STSC", i)] <- stsc_results$Forecasts$Variance_Forecasts
    se[, paste0("STSC", i)] <- (y - stsc_results$Forecasts$Point_Forecasts) ** 2
    crps_score[, paste0("STSC", i)] <- crps(y, stsc_results$Forecasts$Point_Forecasts,
                                            sqrt(stsc_results$Forecasts$Variance_Forecasts), NULL, "normal")$crps

    # Remove & Clean Memory
    rm(list = c("stsc_results"))

    # Update
    i <- i + 1
  }

  ####### STSC - No Forgetting #######
  # Set TV-C-Parameter
  lambda_grid_nf <- 1.00
  kappa_grid_nf <- 1.00
  n_tvc_nf <- (ncol(X) + ncol(Ext_F)) * 1 * 1
  psi_grid_nf <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc_nf / 4)))

  # Apply Function
  stsc_results <- hdflex::stsc(y,
                               X,
                               Ext_F,
                               init,
                               lambda_grid_nf,
                               kappa_grid_nf,
                               bias,
                               gamma_grid,
                               psi_grid_nf,
                               delta,
                               burn_in,
                               burn_in_dsc,
                               metric,
                               equal_weight,
                               incl,
                               parallel,
                               n_threads,
                               NULL)

  # Assign Results
  preds[, "STSC_NF"] <- stsc_results$Forecasts$Point_Forecasts
  vari[, "STSC_NF"] <- stsc_results$Forecasts$Variance_Forecasts
  se[, "STSC_NF"] <- (y - stsc_results$Forecasts$Point_Forecasts) ** 2
  crps_score[, "STSC_NF"] <- crps(y, stsc_results$Forecasts$Point_Forecasts,
                                  sqrt(stsc_results$Forecasts$Variance_Forecasts), NULL, "normal")$crps

  # Remove & Clean Memory
  rm(list = c("stsc_results"))

  ######### --------------------------------------------
  ### Part 3: Evaluation ###
  ####### OOS-Period #######
  # Define Evaluation Period (OOS-Period)
  oos_idx <- which(tdates >= eval_start & tdates <= eval_end)

  # OOS-Data
  oos_y <- y[oos_idx, ]
  oos_preds <- preds[oos_idx, ]
  oos_vari <- vari[oos_idx, ]
  oos_se <- se[oos_idx, ]
  oos_crps <- crps_score[oos_idx, ]

  # Check for Missing Values
  for (obj in list(oos_y, oos_preds, oos_vari, oos_se)) {
    if (any(is.na(obj))) {
      rlang::abort("Error: Missing Values in one of the OOS objects")
    }
  }

  ####### Mean-Squared-Errors #######
  mse <- colMeans(oos_se)
  mse <- matrix(mse, nrow = 1, ncol = length(benchmark_names),
                dimnames = list(y_signal, benchmark_names))

  ####### Mean-Continuous Ranked Probability Score #######
  mcrps <- colMeans(oos_crps)
  mcrps <- matrix(mcrps, nrow = 1, ncol = length(benchmark_names),
                  dimnames = list(y_signal, benchmark_names))

  ######### --------------------------------------------
  ### Save  Results
  write_rds(oos_y, glue("{path}/Results/ES2/{setting}_y_{y_signal}.rds"))
  write_rds(oos_preds, glue("{path}/Results/ES2/{setting}_preds_{y_signal}.rds"))
  write_rds(oos_vari, glue("{path}/Results/ES2/{setting}_vari_{y_signal}.rds"))
  write_rds(oos_se, glue("{path}/Results/ES2/{setting}_se_{y_signal}.rds"))
  write_rds(oos_crps, glue("{path}/Results/ES2/{setting}_crps_{y_signal}.rds"))
  write_rds(list(mse = mse, crps = mcrps), glue("{path}/Results/ES2/{setting}_res_{y_signal}.rds"))

  # Return Results
  return(list(mse = mse, crps = mcrps))
  ######### --------------------------------------------
})

# Stop Cluster
parallel::stopCluster(cl)

# Mean-Squared-Errors
res_mse <- do.call("rbind", purrr::map(res, 1))
round(res_mse / res_mse[, "STSC"], 2)

# Continuous Ranked Probability Score
res_crps <- do.call("rbind", purrr::map(res, 2))
round(res_crps / res_crps[, "STSC"], 2)
######### --------------------------------------------

## 4) Inlcusion TVP-Var Model

In [ ]:
######### --------------------------------------------
### Predict Quarterly Inflation -- Inclusion TVP-VAR ###
# Setting
setting <- "tvp_var"

# Set Global Parameter
cores <- 4
window_size <- 15 * 4
eval_start <- "1991-04-01"
eval_end <- "2021-12-31"

# Set Target-Variable-Names
target_names <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL")

# Set up Parallel-Backend
cl <- parallel::makeCluster(cores)
parallel::clusterExport(cl = cl, varlist = c("setting",
                                             "target_names",
                                             "path",
                                             "window_size",
                                             "eval_start",
                                             "eval_end"),
                        envir = environment())
parallel::clusterEvalQ(cl, {
  library("tidyverse")
  library("magrittr")
  library("purrr")
  library("glue")
  library("rlang")
  library("readr")
  library("doParallel")
  library("hdflex")
})
pbo <- pboptions(type = "timer")

######### --------------------------------------------
### Parallel Loop over Target-Variables ###
res <- pblapply(cl = cl, X = seq_along(target_names), FUN = function(p) {

  ### Load Custom Functions
  source(glue("{path}/Code/_fmodels.R"), local = TRUE)
  source(glue("{path}/Code/_helpers.R"), local = TRUE)

  ######### --------------------------------------------
  ### Build Dataset
  # Response-Name
  y_target <- glue("{target_names[p]}_h1")
  y_signal <- target_names[p]
  not_target <- setdiff(glue("{target_names}_h1"), y_target)

  # Load Y-Data
  y_data <- read_rds(glue("{path}/Data/ES2/P_Signals/y_data.rds")) %>%
    select(-!!not_target)

  # Load Y-Lags
  y_lags <- read_rds(glue("{path}/Data/ES2/P_Signals/y_lags.rds")) %>%
    select(-starts_with(!!not_target))

  # Load P-Signals (Koop & Korobilis 2013 Signals)
  var_sigs <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL", "GDPC1", "FEDFUNDS", "PPIACO", "TOTRESNS", "SP500", "M2REAL", "DPIC96", "PCECC96", "INDPRO", "CUMFNS", "UNRATE", "HOUST", "WPSFD4111", "CES3000000008x", "M1REAL", "OILPRICEx", "GS10", "EXUSUKx", "GPDIC1", "PAYEMS", "IPMANSICS", "AMDMNOx", "OPHPBS")
  x_data <- read_rds(glue("{path}/Data/ES2/P_Signals/x_data.rds")) %>%
    select(Date, !!var_sigs) %>%
    select(-!!y_signal)

  # Load F-Signals
  refi_signals <- read_rds(glue("{path}/Data/ES2/F_Signals/Forecasts_{y_signal}.rds")) %>%
    select(Date,
           TVP_VAR)

  # Combine, lag and prepare data
  data <- y_data %>%
    dplyr::left_join(y_lags, by = "Date") %>%
    dplyr::left_join(x_data, by = "Date") %>%
    dplyr::mutate(across(-c(1, 2, 3, 4), dplyr::lag)) %>%
    dplyr::left_join(refi_signals, by = "Date") %>%
    dplyr::slice(-c(1:3)) %>%
    tibble::column_to_rownames("Date")

  # Assign Response-Variable
  y <- data %>%
    dplyr::select(all_of(y_target)) %>%
    as.matrix()

  # Assign P-Signals
  X <- data %>%
    dplyr::select(all_of(c(colnames(y_lags)[-1], colnames(x_data)[-1]))) %>%
    as.matrix()

  # Assign F-Signals
  Ext_F <- data %>%
    dplyr::select(all_of(colnames(refi_signals)[-1])) %>%
    as.matrix()

  # Dates
  tdates <- rownames(y)

  # Number of Observations
  tlength <- length(tdates)

  # NA-Values
  first_complete <- which(complete.cases(cbind(X, Ext_F)))[1]

  # Remove
  rm(list = c("data", "y_data",  "not_target",
              "y_lags", "x_data", "refi_signals"))

  ######### --------------------------------------------
  ### Build Result Matrices
  # Benchmark Methods
  benchmark_names <- c("AR2", "STAR2",
                       #---#
                       "ST_TVP_VAR",
                       "STSC",
                       "STSC_EXF")

  # Result-Object: Point Forecasts
  preds <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                  dimnames = list(tdates, benchmark_names))

  # Result-Object: Variances
  vari <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                 dimnames = list(tdates, benchmark_names))

  # Result-Object: Squared Errors
  se <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
               dimnames = list(tdates, benchmark_names))

  # Result-Object: Continuous Ranked Probability Score
  crps_score <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                       dimnames = list(tdates, benchmark_names))

  ######### --------------------------------------------
  ### Part 0: Benchmark ###
  ### Rolling AR(2)-Model
  # Time Sequence
  t_seq <- seq(window_size, nrow(y) - 1)

  # Rolling Loop
  for (t in t_seq) {

    ### Split Data ###
    # Train Data
    x_train <- X[(t - window_size + 1):t, 1:2, drop = FALSE]
    y_train <- y[(t - window_size + 1):t, , drop = FALSE]

    # Predict Data
    x_pred <- X[t + 1, 1:2, drop = FALSE]
    y_pred <- y[t + 1, ]

    # Fit and Predict
    pred_ar2 <- ar_model(x_train,
                         y_train,
                         x_pred,
                         2)
    # Save
    preds[t + 1, "AR2"] <- pred_ar2
    vari[t + 1, "AR2"] <- var(y_train)
    se[t + 1, "AR2"] <- (y_pred - pred_ar2) ** 2
  }

  ### Density Transformation (ST-AR2) ###
  # Apply Function
  star2_results <- hdflex::tvc(y,
                               NULL,
                               preds[, "AR2", drop = FALSE],
                               5 * 4,
                               1.00,
                               0.98,
                               FALSE)

  # Assign STSC-Results
  preds[, "STAR2"] <- star2_results$Forecasts$Point_Forecasts
  vari[, "STAR2"] <- star2_results$Forecasts$Variance_Forecasts
  se[, "STAR2"] <- (y - preds[, "STAR2"]) ** 2
  crps_score[, "STAR2"] <- crps(y,
                                preds[, "STAR2"],
                                sqrt(vari[, "STAR2"]),
                                NULL, "normal")$crps

  # Remove & Clean Memory
  rm(list = c("x_train", "y_train", "x_pred", "y_pred",
              "pred_ar2", "star2_results", "t_seq"))

  ######### --------------------------------------------
  ### Part 1: STSC and Variants ###
  ####### ST-TVP-Var #######
  # Set TV-C-Parameter
  init <- 5 * 4
  lambda_grid <- 1.00
  kappa_grid <- c(0.98)
  bias <- TRUE

  # Apply TV-C-Function
  st_tvpvar_results <- hdflex::tvc(y,
                                   NULL,
                                   Ext_F[, "TVP_VAR", drop = FALSE],
                                   init,
                                   lambda_grid,
                                   kappa_grid,
                                   bias)

  # Assign Results
  pred_st_tvpvar <- st_tvpvar_results$Forecasts$Point_Forecasts
  var_st_sttvpvar <- st_tvpvar_results$Forecasts$Variance_Forecasts

  # Remove & Clean Memory
  rm(list = c("st_tvpvar_results"))

  ####### STSC-Model - TVP-VAR + K&K (2013) Signals #######
  # Set TV-C-Parameter
  init <- 5 * 4
  lambda_grid <- c(0.90, 0.95, 1.00)
  kappa_grid <- c(0.94, 0.96, 0.98)
  burn_in <- first_complete + init / 2
  bias <- TRUE

  # Set DSC-Parameter
  gamma_grid <- c(0.40, 0.50, 0.60, 0.70, 0.80, 0.90,
                  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.00)
  n_tvc <- (ncol(X) + ncol(Ext_F)) * length(lambda_grid) * length(kappa_grid)
  psi_grid <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc / 4)))
  delta <- 0.95
  burn_in_dsc <- 1
  metric <- 5
  equal_weight <- TRUE
  incl <- NULL
  parallel <- FALSE
  n_threads <- NULL

  # Apply STSC-Function
  stsc_results <- hdflex::stsc(y,
                               X,
                               Ext_F,
                               init,
                               lambda_grid,
                               kappa_grid,
                               bias,
                               gamma_grid,
                               psi_grid,
                               delta,
                               burn_in,
                               burn_in_dsc,
                               metric,
                               equal_weight,
                               incl,
                               parallel,
                               n_threads,
                               NULL)

  # Assign STSC-Results
  pred_stsc <- stsc_results$Forecasts$Point_Forecasts
  var_stsc  <- stsc_results$Forecasts$Variance_Forecasts

  # Remove & Clean Memory
  rm(list = c("stsc_results"))

  ####### STSC-Model - K&K (2013) Signals #######
  # Set DSC-Parameter
  n_tvc <- (ncol(X) + 0) * length(lambda_grid) * length(kappa_grid)
  psi_grid <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc / 4)))

  # Apply STSC-Function
  stsc_exf_results <- hdflex::stsc(y,
                                   X,
                                   NULL,
                                   init,
                                   lambda_grid,
                                   kappa_grid,
                                   bias,
                                   gamma_grid,
                                   psi_grid,
                                   delta,
                                   burn_in,
                                   burn_in_dsc,
                                   metric,
                                   equal_weight,
                                   incl,
                                   parallel,
                                   n_threads,
                                   NULL)

  # Assign STSC-Results
  pred_stsc_exf <- stsc_exf_results$Forecasts$Point_Forecasts
  var_stsc_exf  <- stsc_exf_results$Forecasts$Variance_Forecasts

  # Remove & Clean Memory
  rm(list = c("stsc_exf_results"))

  ####### Save Point Forecasts #######
  preds[, "ST_TVP_VAR"] <- pred_st_tvpvar
  preds[, "STSC"] <- pred_stsc
  preds[, "STSC_EXF"] <- pred_stsc_exf

  ###### Save Variances #######
  vari[, "ST_TVP_VAR"] <- var_st_sttvpvar
  vari[, "STSC"] <- var_stsc
  vari[, "STSC_EXF"] <- var_stsc_exf

  ####### Save Squared Errors #######
  se[, "ST_TVP_VAR"] <- (y - pred_st_tvpvar) ** 2
  se[, "STSC"] <- (y - pred_stsc) ** 2
  se[, "STSC_EXF"] <- (y - pred_stsc_exf) ** 2

  ####### Save Continuous Ranked Probability Score #######
  crps_score[, "ST_TVP_VAR"] <- crps(y, pred_st_tvpvar, sqrt(var_st_sttvpvar), NULL, "normal")$crps # nolint
  crps_score[, "STSC"] <- crps(y, pred_stsc, sqrt(var_stsc), NULL, "normal")$crps # nolint
  crps_score[, "STSC_EXF"] <- crps(y, pred_stsc_exf, sqrt(var_stsc_exf), NULL, "normal")$crps # nolint

  # Remove & Clean Memory
  rm(list = c("pred_st_tvpvar", "pred_stsc", "pred_stsc_exf",
              "var_star2",
              "var_st_sttvpvar", "var_stsc", "var_stsc_exf"))
  invisible(gc())

  ######### --------------------------------------------
  ### Part 3: Evaluation ###
  ####### OOS-Period #######
  # Define Evaluation Period (OOS-Period)
  oos_idx <- which(tdates >= eval_start & tdates <= eval_end)

  # OOS-Data
  oos_y <- y[oos_idx, ]
  oos_preds <- preds[oos_idx, ]
  oos_vari <- vari[oos_idx, ]
  oos_se <- se[oos_idx, ]
  oos_crps <- crps_score[oos_idx, ]

  # Check for Missing Values
  for (obj in list(oos_y, oos_preds, oos_vari, oos_se)) {
    if (any(is.na(obj))) {
      rlang::abort("Error: Missing Values in one of the OOS objects")
    }
  }

  ####### Mean-Squared-Errors #######
  mse <- colMeans(oos_se)
  mse <- matrix(mse, nrow = 1, ncol = length(benchmark_names),
                dimnames = list(y_signal, benchmark_names))

  ####### Mean-Continuous Ranked Probability Score #######
  mcrps <- colMeans(oos_crps)
  mcrps <- matrix(mcrps, nrow = 1, ncol = length(benchmark_names),
                  dimnames = list(y_signal, benchmark_names))

  ######### --------------------------------------------
  # Save Results
  write_rds(oos_y, glue("{path}/Results/ES2/{setting}_y_{y_signal}.rds"))
  write_rds(oos_preds, glue("{path}/Results/ES2/{setting}_point_forecasts_{y_signal}.rds"))
  write_rds(oos_vari, glue("{path}/Results/ES2/{setting}_variances_{y_signal}.rds"))
  write_rds(oos_se, glue("{path}/Results/ES2/{setting}_squared_errors_{y_signal}.rds"))
  write_rds(oos_crps, glue("{path}/Results/ES2/{setting}_crps_scores_{y_signal}.rds"))
  write_rds(list(mse = mse, crps = mcrps), glue("{path}/Results/ES2/{setting}_res_{y_signal}.rds"))

  # Return Results
  return(list(mse = mse, crps = mcrps))
  ######### --------------------------------------------
})

# Stop Cluster
parallel::stopCluster(cl)

# Continuous Ranked Probability Score
res_crps <- do.call("rbind", purrr::map(res, 2))
round(res_crps / res_crps[, "STSC"], 2)
######### --------------------------------------------

## 5) AR-Augmented Signals

In [ ]:
######### --------------------------------------------
### Predict Quarterly Inflation -- AR-Augmented Signals ###
# Setting
window_size <- 15 * 4
setting <- "inflarion_ar_augmented"

# Set Global Parameter
cores <- 4
eval_start <- "1991-04-01"
eval_end <- "2021-12-31"

# Set Target-Variable-Names
target_names <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL")

# Set up Parallel-Backend
cl <- parallel::makeCluster(cores)
parallel::clusterExport(cl = cl, varlist = c("setting",
                                             "target_names",
                                             "path",
                                             "window_size",
                                             "eval_start",
                                             "eval_end"),
                        envir = environment())
parallel::clusterEvalQ(cl, {
  library("tidyverse")
  library("magrittr")
  library("purrr")
  library("glue")
  library("rlang")
  library("readr")
  library("doParallel")
  library("glmnet")
  library("xgboost")
  library("lightgbm")
  library("ranger")
  library("eDMA")
  library("hdflex")
})
pbo <- pboptions(type = "timer")

######### --------------------------------------------
### Parallel Loop over Target-Variables ###
res <- pblapply(cl = cl, X = seq_along(target_names), FUN = function(p) {

  ### Load Custom Functions
  source(glue("{path}/Code/_fmodels.R"), local = TRUE)
  source(glue("{path}/Code/_helpers.R"), local = TRUE)

  ######### --------------------------------------------
  ### Build Dataset
  # Response-Name
  y_target <- glue("{target_names[p]}_h1")
  y_signal <- target_names[p]
  not_target <- setdiff(glue("{target_names}_h1"), y_target)

  # Load Y-Data
  y_data <- read_rds(glue("{path}/Data/ES2/P_Signals/y_data.rds")) %>%
    select(-!!not_target)

  # Load Y-Lags
  y_lags <- read_rds(glue("{path}/Data/ES2/P_Signals/y_lags.rds")) %>%
    select(-starts_with(!!not_target))

  # Load P-Signals
  x_data <- read_rds(glue("{path}/Data/ES2/P_Signals/x_data.rds")) %>%
    select(-!!y_signal)

  # Load F-Signals (I)
  cm_signals <- read_rds(glue("{path}/Data/ES2/F_Signals/CM_Forecasts_{y_target}.rds"))

  # Load F-Signals (II)
  refi_signals <- read_rds(glue("{path}/Data/ES2/F_Signals/Forecasts_{y_signal}.rds")) %>%
    select(Date,
           SBAR, UCSV,
           GPR_FAC5, TVD_FAC5, VBDVS_FAC5,
           SSVS_FAC60, VBDVS_FAC60,
           VBDVS_X)

  # Load F-Signals (III)
  ar_signals <- read_rds(glue("{path}/Data/ES2/F_Signals/CM_Forecasts_AR_{y_target}.rds"))

  # Combine, lag and prepare data
  data <- y_data %>%
    dplyr::left_join(y_lags, by = "Date") %>%
    dplyr::left_join(x_data, by = "Date") %>%
    dplyr::mutate(across(-c(1, 2, 3, 4), dplyr::lag)) %>%
    dplyr::left_join(cm_signals, by = "Date") %>%
    dplyr::left_join(refi_signals, by = "Date") %>%
    dplyr::left_join(ar_signals, by = "Date") %>%
    dplyr::slice(-c(1:3)) %>%
    tibble::column_to_rownames("Date")

  # Assign Response-Variable
  y <- data %>%
    dplyr::select(
      all_of(
        y_target
      )
    ) %>%
    as.matrix()

  # Assign P-Signals
  X <- data %>%
    dplyr::select(
      all_of(
        c(
          colnames(y_lags)[-1],
          colnames(x_data)[-1]
        )
      )
    ) %>%
    as.matrix()

  # Assign F-Signals
  Ext_F <- data %>%
    dplyr::select(
      all_of(
        c(
          colnames(cm_signals)[-1],
          colnames(refi_signals)[-1],
          colnames(ar_signals)[-1]
        )
      )
    ) %>%
    as.matrix()

  # Dates and Observations
  tdates <- rownames(y)
  tlength <- length(tdates)

  # Remove
  rm(list = c("data", "y_data",  "not_target",
              "y_lags", "x_data", "cm_signals", "refi_signals"))

  ######### --------------------------------------------
  ### Build Result Matrices
  # Benchmark Methods
  benchmark_names <- c("AR2", "STAR2",
                       "DART", "ReLa", "TRF",
                       "XGB", "TCSR",
                       #---#
                       "STSC", "STSC_EXF",
                       "STSCS10", "STSCSFLEX",
                       "PC5DMA", "PC10DMA", "PC15DMA")

  # Result-Object: Point Forecasts
  preds <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                  dimnames = list(tdates, benchmark_names))

  # Result-Object: Variances
  vari <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                 dimnames = list(tdates, benchmark_names))

  # Result-Object: Squared Errors
  se <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
               dimnames = list(tdates, benchmark_names))

  # Result-Object: Continuous Ranked Probability Score
  crps_score <- matrix(NA, ncol = length(benchmark_names), nrow = tlength,
                       dimnames = list(tdates, benchmark_names))

  ######### --------------------------------------------
  ### Part 1: Benchmark ###
  ### Rolling AR(2)-Model
  # Time Sequence
  t_seq <- seq(window_size, nrow(y) - 1)

  # Rolling Loop
  for (t in t_seq) {

    ### Split Data ###
    # Train Data
    x_train <- X[(t - window_size + 1):t, 1:2, drop = FALSE]
    y_train <- y[(t - window_size + 1):t, , drop = FALSE]

    # Predict Data
    x_pred <- X[t + 1, 1:2, drop = FALSE]
    y_pred <- y[t + 1, ]

    # Fit and Predict
    pred_ar2 <- ar_model(x_train,
                         y_train,
                         x_pred,
                         2)
    # Save
    preds[t + 1, "AR2"] <- pred_ar2
    vari[t + 1, "AR2"] <- var(y_train)
    se[t + 1, "AR2"] <- (y_pred - pred_ar2) ** 2
  }

  ### Density Transformation (ST-AR2) ###
  # Apply Function
  star2_results <- hdflex::tvc(y,
                               NULL,
                               preds[, "AR2", drop = FALSE],
                               5 * 4,
                               1.00,
                               0.98,
                               FALSE)

  # Assign STSC-Results
  preds[, "STAR2"] <- star2_results$Forecasts$Point_Forecasts
  vari[, "STAR2"] <- star2_results$Forecasts$Variance_Forecasts
  se[, "STAR2"] <- (y - preds[, "STAR2"]) ** 2
  crps_score[, "STAR2"] <- crps(y,
                                preds[, "STAR2"],
                                sqrt(vari[, "STAR2"]),
                                NULL, "normal")$crps

  # Remove & Clean Memory
  rm(list = c("x_train", "y_train", "x_pred", "y_pred",
              "pred_ar2", "star2_results", "t_seq"))

  ######### --------------------------------------------
  ### Part 2: Forecasting Models with Rolling Window ###
  # Combine, remove NA-rows and subset response
  S_sub <- na.omit(cbind(X, Ext_F))
  y_sub <- y[rownames(S_sub), , drop = FALSE]

  # Get Start Point for Training
  train_start <- sum(rownames(S_sub) < eval_start)
  if (train_start < window_size) {
    rlang::abort("Dataset is too small for the given window size.")
  }

  # First Complete Observation
  first_complete <- which(complete.cases(cbind(X, Ext_F)))[1]
  adj <- first_complete - 1

  # Time Sequence
  t_seq <- seq(train_start,  nrow(S_sub) - 1)

  # Loop with Rolling Window
  for (t in t_seq) {

    ### Split Data ###
    # Train Data
    s_train <- S_sub[(t - window_size + 1):t, , drop = FALSE]
    y_train <- y_sub[(t - window_size + 1):t, , drop = FALSE]

    # Predict Data
    s_pred <- S_sub[t + 1, , drop = FALSE]
    y_pred <- y_sub[t + 1, ]

    ####### Model 1: Relaxed-Lasso #######
    # Parameter
    rela_folds <- 5

    # Fit and Predict
    pred_relax <- relasso_model(s_train,
                                y_train,
                                s_pred,
                                rela_folds,
                                t)

    ####### Model 2: XGBoost #######
    # Parameter
    xgb_folds  <- 5
    xgb_ntrees <- 500
    xgb_lr     <- 0.1
    xgb_target <- NULL
    xgb_cores  <- 1

    # Predict
    pred_xgb <- xgb_model(s_train,
                          y_train,
                          s_pred,
                          xgb_folds,
                          xgb_ntrees,
                          xgb_lr,
                          xgb_target,
                          xgb_cores,
                          t)

    ####### Model 3: LightGBM #######
    # Parameter
    dart_folds  <- 5
    dart_ntrees <- 500
    dart_lr     <- 0.1
    dart_dr     <- 0.1
    dart_target <- NULL
    dart_cores  <- 1

    # Fit and Predict
    pred_dart <- dart_model(s_train,
                            y_train,
                            s_pred,
                            dart_folds,
                            dart_ntrees,
                            dart_lr,
                            dart_dr,
                            dart_target,
                            dart_cores,
                            t)

    ####### Model 4: (Targeted) RandomForests #######
    # Parameter
    trf_ntarget   <- 50
    trf_ntrees    <- 500
    trf_max_depth <- 3
    trf_cores     <- 1

    # Fit and Predict
    pred_trf <- trf_model(s_train,
                          y_train,
                          s_pred,
                          trf_ntarget,
                          trf_ntrees,
                          trf_max_depth,
                          trf_cores,
                          t)

    ####### Model 5: (Targeted) Complete Subset #######
    # Parameter
    csr_n_target <- 20
    csr_n_subset <- 10
    csr_ubound   <- 10000
    csr_sampling <- TRUE

    # Fit and Predict
    pred_csr <- csr_model(s_train,
                          y_train,
                          s_pred,
                          csr_n_target,
                          csr_n_subset,
                          csr_ubound,
                          csr_sampling)

    ####### Save Point Forecasts #######
    preds[t + 1 + adj, "ReLa"] <- pred_relax
    preds[t + 1 + adj, "XGB"]  <- pred_xgb
    preds[t + 1 + adj, "DART"] <- pred_dart
    preds[t + 1 + adj, "TRF"]  <- pred_trf
    preds[t + 1 + adj, "TCSR"] <- pred_csr

    ###### Save Variances #######
    vari[t + 1 + adj, "ReLa"] <- var(y_train)
    vari[t + 1 + adj, "XGB"]  <- var(y_train)
    vari[t + 1 + adj, "DART"] <- var(y_train)
    vari[t + 1 + adj, "TRF"]  <- var(y_train)
    vari[t + 1 + adj, "TCSR"] <- var(y_train)

    ####### Save Squared Errors #######
    se[t + 1 + adj, "ReLa"] <- (y_pred - pred_relax) ** 2
    se[t + 1 + adj, "XGB"]  <- (y_pred - pred_xgb) ** 2
    se[t + 1 + adj, "DART"] <- (y_pred - pred_dart) ** 2
    se[t + 1 + adj, "TRF"]  <- (y_pred - pred_trf) ** 2
    se[t + 1 + adj, "TCSR"] <- (y_pred - pred_csr) ** 2
  }

  # Remove & Clean Memory
  rm(list = c("s_train", "y_train", "s_pred", "y_pred", "S_sub",
              "pred_relax", "pred_xgb", "pred_dart",
              "pred_trf", "pred_csr",
              "t_seq"))
  invisible(gc())

  ######### --------------------------------------------
  ### Part 2: STSC and Variants ###
  ####### STSC-Model #######
  # Set TV-C-Parameter
  init <- 5 * 4
  lambda_grid <- c(0.90, 0.95, 1.00)
  kappa_grid <- c(0.94, 0.96, 0.98)
  bias <- TRUE

  # Set DSC-Parameter
  gamma_grid <- c(0.40, 0.50, 0.60, 0.70, 0.80, 0.90,
                  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.00)
  n_tvc <- (ncol(X) + ncol(Ext_F)) * length(lambda_grid) * length(kappa_grid)
  psi_grid <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc / 4)))
  delta <- 0.95
  burn_in <- first_complete + init / 2
  burn_in_dsc <- 1
  metric <- 5
  equal_weight <- TRUE
  incl <- NULL
  parallel <- FALSE
  n_threads <- NULL

  # Apply Function
  stsc_results <- hdflex::stsc(y,
                               X,
                               Ext_F,
                               init,
                               lambda_grid,
                               kappa_grid,
                               bias,
                               gamma_grid,
                               psi_grid,
                               delta,
                               burn_in,
                               burn_in_dsc,
                               metric,
                               equal_weight,
                               incl,
                               parallel,
                               n_threads,
                               NULL)

  # Assign STSC-Results
  pred_stsc <- stsc_results$Forecasts$Point_Forecasts
  var_stsc <- stsc_results$Forecasts$Variance_Forecasts
  chosen_gamma <- stsc_results$Tuning_Parameters$Gamma
  chosen_psi <- stsc_results$Tuning_Parameters$Psi
  chosen_signals <- stsc_results$Tuning_Parameters$Signals
  chosen_lambda <- stsc_results$Tuning_Parameters$Lambda
  chosen_kappa <- stsc_results$Tuning_Parameters$Kappa

  # Add dates
  names(pred_stsc) <- tdates
  names(var_stsc) <- tdates
  names(chosen_gamma) <- tdates
  names(chosen_psi) <- tdates
  rownames(chosen_signals) <- tdates
  rownames(chosen_lambda) <- tdates
  rownames(chosen_kappa) <- tdates

  # Remove & Clean Memory
  rm(list = c("stsc_results"))

  ####### STSC - Excluding-F-Signals #######
  n_tvc_exf <- (ncol(X) + 0) * length(lambda_grid) * length(kappa_grid)
  psi_grid_exf <- c(1:100, sapply(1:4, function(i) floor(i * n_tvc_exf / 4)))

  # Apply Function
  stsc_exf_results <- hdflex::stsc(y,
                                   X,
                                   NULL,
                                   init,
                                   lambda_grid,
                                   kappa_grid,
                                   bias,
                                   gamma_grid,
                                   psi_grid_exf,
                                   delta,
                                   burn_in,
                                   burn_in_dsc,
                                   metric,
                                   equal_weight,
                                   incl,
                                   parallel,
                                   n_threads,
                                   NULL)

  # Assign Results
  pred_stsc_exf <- stsc_exf_results$Forecasts$Point_Forecasts
  var_stsc_exf  <- stsc_exf_results$Forecasts$Variance_Forecasts

  # Remove & Clean Memory
  rm(list = c("stsc_exf_results"))

  ####### STSCSx - Selecting 10 Forecasts with LASSO #######
  # Set TV-C-Parameter
  init <- 5 * 4
  lambda_grid <- c(0.90, 0.95, 1.00)
  kappa_grid <- c(0.94, 0.96, 0.98)
  bias <- TRUE
  n_cores <- 1

  # Set Lasso-Target-Parameter
  stscsx_target <- 10

  # Apply STSC-S-X Function
  stscsx_results <- stscsx(y,
                           X,
                           Ext_F,
                           lambda_grid,
                           kappa_grid,
                           init,
                           bias,
                           n_cores,
                           window_size,
                           stscsx_target)

  # Assign Results
  pred_stscsx <- stscsx_results[, 1]
  var_stscsx  <- stscsx_results[, 2]

  # Remove & Clean Memory
  rm(list = c("stscsx_results"))

  ####### STSC - Using LASSO to dynamically select forecasts #######
  # Set TV-C-Parameter
  init <- 5 * 4
  lambda_grid <- c(0.90, 0.95, 1.00)
  kappa_grid <- c(0.94, 0.96, 0.98)
  bias <- TRUE
  n_cores <- 1

  # Set Lasso-CV-Parameter
  stscsflex_folds <- 5

  # Apply STSC-S-FLEX Function
  stscsflex_results <- stscsflex(y,
                                 X,
                                 Ext_F,
                                 lambda_grid,
                                 kappa_grid,
                                 init,
                                 bias,
                                 n_cores,
                                 window_size,
                                 stscsflex_folds)

  # Assign Results
  pred_stscsflex <- stscsflex_results[, 1]
  var_stscsflex  <- stscsflex_results[, 2]

  # Remove & Clean Memory
  rm(list = c("stscsflex_results"))

  ####### (Principal Component) Dynamic Model Averaging #######
  # Set Parameter
  pcdma_win    <- window_size
  pcdma_comp   <- 3
  pcdma_alpha  <- 0.99
  pcdma_lambda <- c(0.90, 0.95, 1.00)
  pcdma_kappa  <- 0.98
  pcdma_cores  <- 1
  pcdma_excl   <- c(1, 2)

  # Apply PCA-DMA
  pc5dma_results <- pcdma(y,
                          X,
                          Ext_F,
                          pcdma_win,
                          pcdma_comp,
                          pcdma_alpha,
                          pcdma_lambda,
                          pcdma_kappa,
                          pcdma_cores,
                          pcdma_excl)

  # Assign Results
  pred_pc5dma <- pc5dma_results[, 1]
  var_pc5dma  <- pc5dma_results[, 3]
  crps_pc5dma <- pc5dma_results[, 4]

  # Remove & Clean Memory
  rm(list = c("pc5dma_results"))

  # Set Parameter
  pcdma_comp <- 8

  # Apply PCA-DMA
  pc10dma_results <- pcdma(y,
                           X,
                           Ext_F,
                           pcdma_win,
                           pcdma_comp,
                           pcdma_alpha,
                           pcdma_lambda,
                           pcdma_kappa,
                           pcdma_cores,
                           pcdma_excl)

  # Assign Results
  pred_pc10dma <- pc10dma_results[, 1]
  var_pc10dma  <- pc10dma_results[, 3]
  crps_pc10dma <- pc10dma_results[, 4]

  # Remove & Clean Memory
  rm(list = c("pc10dma_results"))

  # Set Parameter
  pcdma_comp <- 13

  # Apply PCA-DMA
  pc15dma_results <- pcdma(y,
                           X,
                           Ext_F,
                           pcdma_win,
                           pcdma_comp,
                           pcdma_alpha,
                           pcdma_lambda,
                           pcdma_kappa,
                           pcdma_cores,
                           pcdma_excl)

  # Assign Results
  pred_pc15dma <- pc15dma_results[, 1]
  var_pc15dma  <- pc15dma_results[, 3]
  crps_pc15dma <- pc15dma_results[, 4]

  # Remove & Clean Memory
  rm(list = c("pc15dma_results"))

  ######### --------------------------------------------
  ####### Save Point Forecasts #######
  preds[, "STSC"] <- pred_stsc
  preds[, "STSC_EXF"] <- pred_stsc_exf
  preds[, "STSCS10"] <- pred_stscsx
  preds[, "STSCSFLEX"] <- pred_stscsflex
  preds[, "PC5DMA"] <- pred_pc5dma
  preds[, "PC10DMA"] <- pred_pc10dma
  preds[, "PC15DMA"] <- pred_pc15dma

  ####### Save Variances #######
  vari[, "STSC"] <- var_stsc
  vari[, "STSC_EXF"] <- var_stsc_exf
  vari[, "STSCS10"] <- var_stscsx
  vari[, "STSCSFLEX"] <- var_stscsflex
  vari[, "PC5DMA"] <- var_pc5dma
  vari[, "PC10DMA"] <- var_pc10dma
  vari[, "PC15DMA"] <- var_pc15dma

  ####### Save Squared Errors #######
  se[, "STSC"] <- (y - pred_stsc) ** 2
  se[, "STSC_EXF"] <- (y - pred_stsc_exf) ** 2
  se[, "STSCS10"] <- (y - pred_stscsx) ** 2
  se[, "STSCSFLEX"] <- (y - pred_stscsflex) ** 2
  se[, "PC5DMA"] <- (y - pred_pc5dma) ** 2
  se[, "PC10DMA"] <- (y - pred_pc10dma) ** 2
  se[, "PC15DMA"] <- (y - pred_pc15dma) ** 2

  ####### Save Continuous Ranked Probability Score #######
  crps_score[, "STSC"] <- crps(y, pred_stsc, sqrt(var_stsc), NULL, "normal")$crps # nolint
  crps_score[, "STSC_EXF"] <- crps(y, pred_stsc_exf, sqrt(var_stsc_exf), NULL, "normal")$crps # nolint
  crps_score[, "STSCS10"] <- crps(y, pred_stscsx, sqrt(var_stscsx), NULL, "normal")$crps # nolint
  crps_score[, "STSCSFLEX"] <- crps(y, pred_stscsflex, sqrt(var_stscsflex), NULL, "normal")$crps # nolint
  crps_score[, "PC5DMA"]  <- crps_pc5dma
  crps_score[, "PC10DMA"] <- crps_pc10dma
  crps_score[, "PC15DMA"] <- crps_pc15dma

  # Remove & Clean Memory
  rm(list = c("pred_stsc", "pred_stsc_exf",
              "pred_stscsx", "pred_stscsflex",
              "pred_pc5dma", "pred_pc10dma", "pred_pc15dma",
              "var_stsc", "var_stsc_exf",
              "var_stscsflex", "var_stscsx",
              "var_pc5dma", "var_pc10dma", "var_pc15dma",
              "crps_pc5dma", "crps_pc10dma", "crps_pc15dma"))
  invisible(gc())

  ######### --------------------------------------------
  ### Part 3: Evaluation ###
  ####### OOS-Period #######
  # Define Evaluation Period (OOS-Period)
  oos_idx <- which(tdates >= eval_start & tdates <= eval_end)

  # OOS-Data
  oos_y <- y[oos_idx, ]
  oos_preds <- preds[oos_idx, ]
  oos_vari <- vari[oos_idx, ]
  oos_se <- se[oos_idx, ]
  oos_crps <- crps_score[oos_idx, ]
  oos_chosen_gamma <- chosen_gamma[oos_idx]
  oos_chosen_psi <- chosen_psi[oos_idx]
  oos_chosen_signals <- chosen_signals[oos_idx, ]
  oos_chosen_lambda <- chosen_lambda[oos_idx, ]
  oos_chosen_kappa <- chosen_kappa[oos_idx, ]

  # Check for Missing Values
  for (obj in list(oos_y, oos_preds, oos_vari, oos_se)) {
    if (any(is.na(obj))) {
      rlang::abort("Error: Missing Values in one of the OOS objects")
    }
  }

  ####### Mean-Squared-Errors #######
  mse <- colMeans(oos_se)
  mse <- matrix(mse, nrow = 1, ncol = length(benchmark_names),
                dimnames = list(y_signal, benchmark_names))

  ####### Mean-Continuous Ranked Probability Score #######
  mcrps <- colMeans(oos_crps)
  mcrps <- matrix(mcrps, nrow = 1, ncol = length(benchmark_names),
                  dimnames = list(y_signal, benchmark_names))

  ######### --------------------------------------------
  # Save Results
  write_rds(oos_y, glue("{path}/Results/ES2/{setting}_y_{y_signal}.rds"))
  write_rds(oos_preds, glue("{path}/Results/ES2/{setting}_point_forecasts_{y_signal}.rds"))
  write_rds(oos_vari, glue("{path}/Results/ES2/{setting}_variances_{y_signal}.rds"))
  write_rds(oos_se, glue("{path}/Results/ES2/{setting}_squared_errors_{y_signal}.rds"))
  write_rds(oos_crps, glue("{path}/Results/ES2/{setting}_crps_scores_{y_signal}.rds"))
  write_rds(list(mse = mse, crps = mcrps), glue("{path}/Results/ES2/{setting}_res_{y_signal}.rds"))

  # Save STSC-Parameter
  write_rds(list(
    gamma = oos_chosen_gamma,
    psi = oos_chosen_psi,
    signals = oos_chosen_signals,
    lambda = oos_chosen_lambda,
    kappa = oos_chosen_kappa
  ), glue("{path}/Results/ES2/{setting}_stsc_params_{y_signal}.rds"))

  # Return Results
  return(list(mse = mse, crps = mcrps))
  ######### --------------------------------------------
})

# Stop Cluster
parallel::stopCluster(cl)

# Mean-Squared-Errors
res_mse <- do.call("rbind", purrr::map(res, 1))
round(res_mse / res_mse[, "STSC"], 2)

# Continuous Ranked Probability Score
res_crps <- do.call("rbind", purrr::map(res, 2))
round(res_crps / res_crps[, "STSC"], 2)
######### --------------------------------------------

## 6) Plots

In [6]:
######### --------------------------------------------
### Global Settings
# Names
plot_name <- c("GDP Deflator", "PCE Deflator", "Total CPI", "Core CPI")
target_names <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL")
setting <- "inflation_60

# Store reusable parameters
black_color <- "#000000"
grey_color <- "#807f7f"
orange_color <- "#E69F00"
darkblue_color <- "#3a5795"
sky_color <- "#56B4E9"
green_color <- "#009E73"
blue_color <- "#0072B2"
red_color <- "#D55E00"
purple_color <- "#CC79A7"
dotted_line_color <- "#7b7979da"
lightgrey_color <- "#807f7f18"
text_size <- 24
axis_text_size <- 20
legend_text_size <- 20
legend_key_size <- unit(4, "line")
x_breaks <- seq(ymd("1995-01-01"), ymd("2021-12-01"), by = "5 year")
x_expand <- expansion(mult = c(0, 0.01))
y_expand <- expansion(mult = c(0.05, 0.05))
hline_size <- 0.5
vline_size <- 0.5

# Set the theme globally
theme_set(
  theme_bw() +
    theme(
      plot.title = element_text(size = text_size,
                                face = "bold",
                                hjust = 0.5,
                                margin = margin(0, 0, 10, 0)),
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(),
      text = element_text(size = text_size),
      axis.text = element_text(size = axis_text_size),
      axis.text.x = element_text(vjust = 0.5),
      axis.title.x = element_text(vjust = -0.5),
      axis.title = element_text(size = text_size),
      strip.text = element_text(size = text_size),
      legend.title = element_blank(),
      legend.text = element_text(size = legend_text_size),
      legend.position = "bottom",
      legend.margin = margin(t = -30),
      legend.key.size = legend_key_size
    )
)

# Common Plot Asthetics
plot_aesthetics <- list(
  scale_x_date(breaks = x_breaks, date_labels = "%Y", expand = x_expand),
  guides(color = guide_legend(override.aes = list(size = 6,
                                                  alpha = 1,
                                                  linewidth = 1.25)))
)
######### --------------------------------------------

In [7]:
######### --------------------------------------------
# Loop over Target-Variables
for (p in 1:4) {

  ### Response
  y_target <- glue::glue("{target_names[p]}_h1")
  y_signal <- target_names[p]
  not_target <- setdiff(glue::glue("{target_names}_h1"), y_target)

  ### Plot: Cumulative Squared Error Differences (CSSED) ###
  # Load CSSED
  df <- read_rds(glue::glue("{path}/Results/ES2/{setting}_squared_errors_{y_signal}.rds")) %>%
    as_tibble(rownames = "Date") %>%
    mutate(Date = ymd(Date)) %>%
    mutate(across(-Date, ~AR2 - .)) %>%
    mutate(across(-Date, ~cumsum(.))) %>%
    dplyr::select(-AR2, -STAR2) %>%
    pivot_longer(!Date, names_to = "Model", values_to = "value") %>%
    mutate(Method = ifelse(Model == "STSC", "STSC", "Benchmark Methods")) %>%
    mutate(Method = factor(Method, levels = c("STSC", "Benchmark Methods")))

  # Plot CSSED
  plot_cssed <- df %>%
    ggplot() +
    geom_line(aes(Date,
                  value,
                  group = Model,
                  color = Method,
                  linetype = Method,
                  linewidth = Method,
                  alpha = Method)) +
    labs(title = "",
         x = "",
         y = "Cum. Squared Error Differences") +
    scale_y_continuous(limit = c(-0.0008, 0.0011),
                       expand = y_expand) +
    scale_linetype_manual(values = c(1, 2)) +
    scale_linewidth_manual(values = c(1.0, 0.5),
                           guide = "none") +
    scale_alpha_manual(values = c(1, 0.5),
                       guide = "none")  +
    scale_color_manual(values = c(black_color, grey_color)) +
    geom_hline(yintercept = 0,
               linetype = "dotted",
               color = dotted_line_color,
               linewidth = hline_size) +
    theme(
      axis.text.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.title.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.ticks.y = element_line(colour = ifelse(p %in% c(1, 3), "black", "white"))
    ) +
    plot_aesthetics

  ### Plot: Cumulative CRPS Differences ###
  # Load CRPS
  df <- read_rds(glue("{path}/Results/ES2/{setting}_crps_scores_{y_signal}.rds")) %>%
    as_tibble(rownames = "Date") %>%
    mutate(Date = ymd(Date)) %>%
    mutate(across(-Date, ~STAR2 - .)) %>%
    mutate(across(-Date, ~cumsum(.))) %>%
    dplyr::select(-AR2, -STAR2) %>%
    pivot_longer(!Date, names_to = "Model", values_to = "value") %>%
    mutate(Method = ifelse(Model == "STSC", "STSC", "Benchmark Methods")) %>%
    mutate(Method = factor(Method, levels = c("STSC", "Benchmark Methods")))

  # Plot CRPS
  plot_crps <- df %>%
    ggplot() +
    geom_line(aes(Date,
                  value,
                  group = Model,
                  color = Method,
                  linetype = Method,
                  linewidth = Method,
                  alpha = Method)) +
    labs(title = glue("{plot_name[p]}"),
         y = "Cum. CRPS Differences",
         x = "") +
    scale_y_continuous(limit = c(-0.05, 0.05),
                       expand = y_expand) +
    scale_linetype_manual(values = c(1, 2)) +
    scale_linewidth_manual(values = c(1.0, 0.5),
                           guide = "none") +
    scale_alpha_manual(values = c(1, 0.5),
                       guide = "none")  +
    scale_color_manual(values = c(black_color, grey_color)) +
    geom_hline(yintercept = 0,
               linetype = "dotted",
               color = dotted_line_color,
               linewidth = hline_size) +
    theme(
      #plot.title = element_text(colour = ifelse(p %in% c(1), "black", "white")),
      axis.text.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.title.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.ticks.y = element_line(colour = ifelse(p %in% c(1, 3), "black", "white"))
    ) +
    plot_aesthetics

  ### Plot: Subset-Size Psi ###
  # Load Psi
  df <- read_rds(glue("{path}/Results/ES2/{setting}_stsc_params_{y_signal}.rds"))$psi %>%
    as_tibble(rownames = "Date") %>%
    mutate(Date = ymd(Date)) %>%
    pivot_longer(!Date, names_to = "Parameter", values_to = "value") %>%
    filter(value > 0) %>%
    mutate(Description = "Subset-Size")

  # Plot Psi
  plot_psi <- df %>%
    mutate(value_factor = factor(value, levels = unique(sort(value)))) %>%
    ggplot() +
    geom_point(aes(Date,
                   value_factor,
                   color = Description),
               size = 1.75, alpha = 1.0, shape = 18) +
    labs(title = "",
         y = expression("Subset Size" ~ bold(psi)),
         x = "") +
    scale_color_manual(values = c(black_color)) +
    scale_y_discrete(
      expand = y_expand,
      limits = factor(c(1:100, 101:109, 1037, 1501:1509, 2074, 3001:3009, 3111, 3301:3309, 4149)),
      breaks = factor(c(1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 1037, 2074, 3111, 4149)),
      labels = factor(c(1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 1037, 2074, 3111, 4149))
    ) +
    theme(
      legend.position = "none",
      plot.title = element_text(colour = ifelse(p %in% c(1), "black", "white")),
      axis.text.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.title.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.ticks.y = element_line(colour = ifelse(p %in% c(1, 3), "black", "white"))
    ) +
    plot_aesthetics

  ### Plot: Selected Predictive Signals ###
  # Load Y-Lags
  lag_names <- read_rds(glue("{path}/Data/ES2/P_Signals/y_lags.rds")) %>%
    select(-Date,
           -starts_with(!!not_target)) %>%
    colnames()

  # Load P-Signals
  x_names <- read_rds(glue("{path}/Data/ES2/P_Signals/x_data.rds")) %>%
    select(-Date,
           -!!y_signal) %>%
    colnames()

  # Load F-Signals (I)
  cm_names <- read_rds(glue("{path}/Data/ES2/F_Signals/CM_Forecasts_{y_target}.rds")) %>%
    select(-Date) %>%
    colnames()

  # Load F-Signals (II)
  f_names <- read_rds(glue("{path}/Data/ES2/F_Signals/Forecasts_{y_signal}.rds")) %>%
    select(SBAR, UCSV,
           GPR_FAC5, TVD_FAC5, VBDVS_FAC5,
           SSVS_FAC60, VBDVS_FAC60,
           VBDVS_X) %>%
    colnames()

  # Chosen Signals
  df <- read_rds(glue("{path}/Results/ES2/{setting}_stsc_params_{y_signal}.rds"))$signals %>%
    as_tibble(rownames = "Date") %>%
    mutate(Date = ymd(Date)) %>%
    mutate(t(replicate(nrow(.), seq_len(ncol(.)))) * pick(where(is.numeric))) %>%
    pivot_longer(!Date, names_to = "Predictor", values_to = "value") %>%
    filter(value > 0) %>%
    mutate(Indicator = case_when(
      Predictor %in% c(lag_names, x_names) ~ "P-Signal",
      Predictor %in% c(f_names, cm_names) ~ "F-Signal",
      TRUE ~ "Unknown"
    )) %>%
    mutate(Indicator = factor(Indicator, levels = c("P-Signal",
                                                    "F-Signal",
                                                    "Unknown")))

  # Plot Signals
  plot_sigs <- df %>%
    ggplot() +
    geom_point(aes(Date, value, color = Indicator),
               size = 1.75, alpha = 0.9, shape = 18) +
    labs(title = "",
         y = "Predictive Signals",
         x = "") +
    scale_y_continuous(labels = c(1, seq(50, 450, 50)),
                       breaks = c(1, seq(50, 450, 50)),
                       limits = c(1, 470),
                       expand = y_expand) +
    scale_color_manual(values = c("P-Signal" = "black", "F-Signal" = sky_color),
                       breaks = c("P-Signal", "F-Signal")) +
    theme(
      plot.title = element_text(colour = ifelse(p %in% c(1), "black", "white")),
      axis.text.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.title.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.ticks.y = element_line(colour = ifelse(p %in% c(1, 3), "black", "white"))
    ) +
    plot_aesthetics

  ### Plot: Forgetting Factor Gamma ###
  # Load Gamma
  df <- read_rds(glue("{path}/Results/ES2/{setting}_stsc_params_{y_signal}.rds"))$gamma %>%
    as_tibble(rownames = "Date") %>%
    mutate(Date = ymd(Date)) %>%
    pivot_longer(!Date, names_to = "Parameter", values_to = "value") %>%
    filter(value != 0) %>%
    mutate(Description = "Discount Factor")

  # Plot Gamma
  plot_gamma <- df %>%
    mutate(value_factor = factor(value, levels = unique(sort(value)))) %>%
    ggplot(aes(Date, value_factor, color = Description)) +
    geom_point(size = 1.75, alpha = 1.0, shape = 18) +
    labs(title = glue("{plot_name[p]}"),
         y = expression("Discount Factor" ~ bold(gamma)),
         x = "") +
    scale_color_manual(values = c(black_color)) + #,
                       #guide = guide_legend(override.aes = list(size = 4,
                       #                                         alpha = 1))) +
    scale_y_discrete(
      expand = y_expand,
      limits = factor(c(0.4, 0.5, 0.6, 0.7, 0.8, 0.9, seq(0.91, 1.00, 0.01))),
      breaks = factor(c(0.4, 0.5, 0.6, 0.7, 0.8, 0.9, seq(0.91, 1.00, 0.01))),
      labels = function(x) sprintf("%.2f", as.numeric(as.character(x)))
    ) +
    theme(
      legend.position = "none",
      axis.text.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.title.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.ticks.y = element_line(colour = ifelse(p %in% c(1, 3), "black", "white"))
    ) +
    plot_aesthetics

  ### Plot: Forgetting Factor Lambda ###
  # Load Lambda
  df <- read_rds(glue("{path}/Results/ES2/{setting}_stsc_params_{y_signal}.rds"))$lambda %>%
    as_tibble(rownames = "Date") %>%
    mutate(Date = ymd(Date)) %>%
    pivot_longer(!Date,
                 names_to = "Parameter",
                 values_to = "value") %>%
    mutate(Description = "Lambda value") %>%
    mutate(Parameter = case_when(
      Parameter == "0.9" ~ "0.90",
      Parameter == "1" ~ "1.00",
      TRUE ~ Parameter
    )) %>%
    group_by(Date) %>%
    mutate(value = value / sum(value) * 100) %>%
    ungroup()

  # Plot Lambda
  plot_lambda <- df %>%
    ggplot(aes(x = Date, y = value, fill = Parameter)) +
    geom_area(position = "stack", alpha = 0.6) +
    labs(title = "",
         y = expression("Discount Factor" ~ bold(lambda)),
         x = "") +
    scale_y_continuous(expand = y_expand,
                       labels = scales::percent_format(scale = 1.0)) +
    scale_fill_manual(values = c("0.90" = "gray80",
                                 "0.95" = "gray50",
                                 "1.00" = "gray20")) +
    theme(
      plot.title = element_text(colour = ifelse(p %in% c(1), "black", "white")),
      axis.text.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.title.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.ticks.y = element_line(colour = ifelse(p %in% c(1, 3), "black", "white")),
      legend.key.size = unit(0.8, "cm")
    ) +
    plot_aesthetics

  ### Plot: Forgetting Factor Kappa ###
  # Load Kappa
  df <- read_rds(glue("{path}/Results/ES2/{setting}_stsc_params_{y_signal}.rds"))$kappa %>%
    as_tibble(rownames = "Date") %>%
    mutate(Date = ymd(Date)) %>%
    pivot_longer(!Date,
                 names_to = "Parameter",
                 values_to = "value") %>%
    mutate(Description = "Kappa value") %>%
    group_by(Date) %>%
    mutate(value = value / sum(value) * 100) %>%
    ungroup()

  # Plot Kappa
  plot_kappa <- df %>%
    ggplot(aes(x = Date, y = value, fill = Parameter)) +
    geom_area(position = "stack", alpha = 0.6) +
    labs(title = "",
         y = expression("Discount Factor" ~ bold(kappa)),
         x = "") +
    scale_y_continuous(expand = y_expand,
                       labels = scales::percent_format(scale = 1.0)) +
    scale_fill_manual(values = c("0.94" = "gray80",
                                 "0.96" = "gray50",
                                 "0.98" = "gray20")) +
    theme(
      plot.title = element_text(colour = ifelse(p %in% c(1), "black", "white")),
      axis.text.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.title.y = element_text(colour = ifelse(p %in% c(1, 3), "black", "white")),
      axis.ticks.y = element_line(colour = ifelse(p %in% c(1, 3), "black", "white")),
      legend.key.size = unit(0.8, "cm")
    ) +
    plot_aesthetics

  # Save
  write_rds(plot_cssed, glue("{path}/Results/ES2/Plots/plot_cssed_{y_signal}.rds"))
  write_rds(plot_crps, glue("{path}/Results/ES2/Plots/plot_crps_{y_signal}.rds"))
  write_rds(plot_psi, glue("{path}/Results/ES2/Plots/plot_psi_{y_signal}.rds"))
  write_rds(plot_sigs, glue("{path}/Results/ES2/Plots/plot_sigs_{y_signal}.rds"))
  write_rds(plot_gamma, glue("{path}/Results/ES2/Plots/plot_gamma_{y_signal}.rds"))
  write_rds(plot_lambda, glue("{path}/Results/ES2/Plots/plot_lambda_{y_signal}.rds"))
  write_rds(plot_kappa, glue("{path}/Results/ES2/Plots/plot_kappa_{y_signal}.rds"))
}
######### --------------------------------------------

In [ ]:
######### --------------------------------------------
### Plot: Combination I ###
# Define file names
file_names <- list(
  c("plot_crps_GDPCTPI.rds", "plot_cssed_GDPCTPI.rds", "plot_psi_GDPCTPI.rds", "plot_sigs_GDPCTPI.rds"),
  c("plot_crps_PCECTPI.rds", "plot_cssed_PCECTPI.rds", "plot_psi_PCECTPI.rds", "plot_sigs_PCECTPI.rds"),
  c("plot_crps_CPIAUCSL.rds", "plot_cssed_CPIAUCSL.rds", "plot_psi_CPIAUCSL.rds", "plot_sigs_CPIAUCSL.rds"),
  c("plot_crps_CPILFESL.rds", "plot_cssed_CPILFESL.rds", "plot_psi_CPILFESL.rds", "plot_sigs_CPILFESL.rds")
)

# Function to read files and create combined plot
create_combined_plot <- function(file_names, path) {
  plots <- lapply(file_names, function(file) {
    readr::read_rds(glue::glue("{path}/Results/ES2/Plots/{file}"))
  })

  cowplot::plot_grid(
    plotlist = plots,
    rel_heights = c(1, 1, 0.925, 1),
    rel_widths = c(1),
    nrow = 4,
    align = "v"
  )
}

# Create combined plots
cplots <- lapply(file_names, create_combined_plot, path = path)

# Combine plots
plot_inf1 <- cowplot::plot_grid(plotlist = cplots[1:2], nrow = 1, align = "v")
plot_inf2 <- cowplot::plot_grid(plotlist = cplots[3:4], nrow = 1, align = "v")

# Show Plot
options(repr.plot.width = 15, repr.plot.height = 25)
plot_inf1
plot_inf2

# Save
ggsave("plot_inflation1.pdf", plot_inf1, path = glue::glue("{path}/Results/ES2/Plots/"), width = 20, height = 30)
ggsave("plot_inflation2.pdf", plot_inf2, path = glue::glue("{path}/Results/ES2/Plots/"), width = 20, height = 30)
######### --------------------------------------------

In [ ]:
######### --------------------------------------------
### Plot: Combination II ###
# Define file names
file_names <- list(
  c("plot_gamma_GDPCTPI.rds", "plot_lambda_GDPCTPI.rds", "plot_kappa_GDPCTPI.rds"),
  c("plot_gamma_PCECTPI.rds", "plot_lambda_PCECTPI.rds", "plot_kappa_PCECTPI.rds"),
  c("plot_gamma_CPIAUCSL.rds", "plot_lambda_CPIAUCSL.rds", "plot_kappa_CPIAUCSL.rds"),
  c("plot_gamma_CPILFESL.rds", "plot_lambda_CPILFESL.rds", "plot_kappa_CPILFESL.rds")
)

# Function to read files and create combined plot
create_combined_plot <- function(file_names, path) {
  plots <- lapply(file_names, function(file) {
    readr::read_rds(glue::glue("{path}/Results/ES2/Plots/{file}"))
  })

  cowplot::plot_grid(
    plotlist = plots,
    rel_heights = c(1, 1, 1),
    rel_widths = 1,
    nrow = 3,
    align = "v"
  )
}

# Create combined plots
cplots <- lapply(file_names, create_combined_plot, path = path)

# Combine plots
plot_inf3 <- cowplot::plot_grid(plotlist = cplots[1:2], nrow = 1, align = "v")
plot_inf4 <- cowplot::plot_grid(plotlist = cplots[3:4], nrow = 1, align = "v")

# Show Plot
options(repr.plot.width = 20, repr.plot.height = 25)
plot_inf3
plot_inf4

# Save
ggsave("plot_inflation3.pdf", plot_inf3, path = glue("{path}/Results/ES2/Plots/"), width = 20, height = 25)
ggsave("plot_inflation4.pdf", plot_inf4, path = glue("{path}/Results/ES2/Plots/"), width = 20, height = 25)
######### --------------------------------------------

In [ ]:
######### --------------------------------------------
### Plot: Calibration of Aggregate Predictive Densities (I) ###
# Setting
setting <- "inflation_60"

# Number of Draws
n <- 5000

# Set up empty tibble
data <- tibble()

# Loop over Target-Variables
for (p in 1:4) {

  # Load Y
  y_signal <- target_names[p]
  y <- read_rds(glue("{path}/Results/ES2/{setting}_y_{y_signal}.rds")) %>%
    as_tibble(rownames = "Date") %>%
    mutate(Date = ymd(Date)) %>%
    rename("y_real" = 2)

  # Load Predictive Density (Mean)
  pred_stsc <- read_rds(glue("{path}/Results/ES2/{setting}_point_forecasts_{y_signal}.rds")) %>%
    as_tibble(rownames = "Date") %>%
    mutate(Date = ymd(Date)) %>%
    rename("mu" = STSC) %>%
    dplyr::select(Date, mu)

  # Load Predictive Density (Variance)
  var_stsc <- read_rds(glue("{path}/Results/ES2/{setting}_variances_{y_signal}.rds")) %>%
    as_tibble(rownames = "Date") %>%
    mutate(Date = ymd(Date)) %>%
    mutate("sd" = sqrt(STSC)) %>%
    dplyr::select(Date, sd)

  # Combine and prepare data
  tmp <- pred_stsc %>%
    left_join(var_stsc, by = "Date") %>%
    left_join(y, by = "Date") %>%
    mutate(qrt = paste0(year(Date), "Q", quarter(Date)),
           Date = quarter(Date, with_year = TRUE)) %>%
    filter(Date %in% c("2021.4", "2017.4",
                       "2013.4", "2009.4",
                       "2005.4", "2001.4",
                       "1997.4", "1993.4")) %>%
    mutate(Date = factor(Date),
           low = mu - 3 * sd,
           high = mu + 3 * sd) %>%
    uncount(n, .id = "row") %>%
    mutate(x = (1 - row / n) * low + row / n * high,
           norm = dnorm(x, mu, sd),
           Type = plot_name[p],
           quantile = pnorm(y_real, mean = mu, sd = sd))

  # Combine
  data  <- bind_rows(data, tmp)
}

# Plot Aggregate Predictive Densities
plot_ridge <- data %>%
  mutate(Type = fct_relevel(Type, c("GDP Deflator",
                                    "PCE Deflator",
                                    "Total CPI",
                                    "Core CPI"))) %>%
  ggplot(aes(x = x,
             y = qrt,
             height = norm)) +
  facet_wrap(~Type, scale = "free_x") +
  ggridges::geom_density_ridges(stat = "identity",
                                alpha = 0.5) +
  geom_point(aes(x = y_real, y = qrt),
             color = "black",
             size = 1.0,
             shape = 4) +
  scale_y_discrete(expand = expansion(mult = c(0.05, 0.19))) +
  labs(title = "",
       y = "",
       x  = "") +
  theme_bw() +
  theme(strip.background = element_rect(fill = alpha("white", 0.15)),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        text = element_text(size = 16),
        axis.text = element_text(size = 12),
        axis.title = element_text(size = 16),
        axis.text.x = element_text(vjust = 0.5),
        axis.title.x = element_text(vjust = -0.5),
        strip.text = element_text(size = 16))

# Show Plot
options(repr.plot.width = 10, repr.plot.height = 9)
plot_ridge

# Save Plot
ggsave("plot_inflation_ridge.pdf",
       plot_ridge,
       path = glue::glue("{path}/Results/ES2/Plots/"),
       width = 10,
       height = 10)
####### --------------------------------------------

In [ ]:
######### --------------------------------------------
### Plot: Calibration of Aggregate Predictive Densities (II) ###
# Load U-Vectors
cdf_t <- read_rds(glue("{path}/Results/ES2/pit_cdf_t.rds")) %>%
  as_tibble() %>%
  pivot_longer(
    everything(),
    names_to = "target_var",
    values_to = "PIT_value"
  ) %>%
  mutate(
    target_var = fct_relevel(target_var, c("GDP Deflator",
                                           "PCE Deflator",
                                           "Total CPI",
                                           "Core CPI"))
  )

# Generate uniform distribution
data_unif <- data.frame(value = seq(0, 1, length.out = 1000))
data_unif$density <- dunif(data_unif$value, min = 0, max = 1)

# Plot Histograms
plot_hist <- cdf_t %>%
  ggplot(aes(x = PIT_value)) +
  facet_wrap(~target_var, scales = "free_x") +
  geom_histogram(aes(y = ..count..), color = "white",
                 alpha = 0.5,
                 binwidth = 0.125,
                 center = 0.0625) +
  scale_y_continuous(expand = y_expand) +
  geom_hline(yintercept = 0, linetype = "dotted", color = "#7b797989") +
  geom_line(data = data_unif,
            aes(x = value, y = density * nrow(cdf_t) / 4 * 0.125),
            color = "blue", linewidth = 0.75, linetype = "dashed") +
  scale_color_manual(values = c("red")) +
  labs(title = "",
       y = "",
       x = "") +
  theme_bw() +
  theme(strip.background = element_rect(fill = alpha("white", 0.15)),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        text = element_text(size = 16),
        axis.text = element_text(size = 12),
        axis.title = element_text(size = 16),
        axis.text.x = element_text(vjust = 0.5),
        axis.title.x = element_text(vjust = -0.5),
        strip.text = element_text(size = 16))

# Sort PIT Values
scdf_t <- cdf_t %>%
  arrange(target_var, PIT_value)

# Confidence Bands (normal approximation)
alpha <- 0.1
z <- qnorm(1 - alpha / 2)
unif_tq <- seq(0, 1, length.out = 123)
l_band <- unif_tq - z * sqrt(unif_tq * (1 - unif_tq) / 123)
u_band <- unif_tq + z * sqrt(unif_tq * (1 - unif_tq) / 123)

# Create tibble
data <- scdf_t %>%
  mutate(
    theoretical_quantiles = rep(unif_tq, times = length(unique(target_var))),
    l_band = rep(l_band, times = length(unique(target_var))),
    u_band = rep(u_band, times = length(unique(target_var)))
  )

# Plot QQ-Plot
plot_qq <- ggplot(data, aes(x = theoretical_quantiles, y = PIT_value)) +
  facet_wrap(~target_var, scales = "free_x") +
  geom_point(color = "black", size = 0.25) +
  geom_abline(intercept = 0, slope = 1, color = "red", linetype = "dashed") +
  geom_ribbon(aes(ymin = l_band, ymax = u_band), alpha = 0.1, fill = "blue") +
  labs(title = "",
       x = "Theoretical Quantiles",
       y = "Empirical Quantiles") +
  theme_bw() +
  theme(strip.background = element_rect(fill = alpha("white", 0.15)),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        text = element_text(size = 16),
        axis.text = element_text(size = 12),
        axis.title = element_text(size = 16),
        axis.text.x = element_text(vjust = 0.5),
        axis.title.x = element_text(vjust = -0.5),
        strip.text = element_text(size = 16))

# Show Plot
options(repr.plot.width = 10, repr.plot.height = 9)
plot_hist
plot_qq

# Save Plot
ggsave("plot_inflation_pit.pdf",
       plot_hist,
       path = glue::glue("{path}/Results/ES2/Plots/"),
       width = 10,
       height = 9)

# Save Plot
ggsave("plot_inflation_qq.pdf",
       plot_qq,
       path = glue::glue("{path}/Results/ES2/Plots/"),
       width = 10,
       height = 9)
######### --------------------------------------------

In [ ]:
######### --------------------------------------------
### Most important predictors
# Loop over target variables
for (p in 1:4) {

  ### Response
  y_target <- glue::glue("{target_names[p]}_h1")
  y_signal <- target_names[p]
  not_target <- setdiff(glue::glue("{target_names}_h1"), y_target)

  # Load Y-Lags
  lag_names <- read_rds(glue("{path}/Data/ES2/P_Signals/y_lags.rds")) %>%
    select(-Date,
           -starts_with(!!not_target)) %>%
    colnames()

  # Load P-Signals
  simple_names <- read_rds(glue("{path}/Data/ES2/P_Signals/x_data.rds")) %>%
    select(-Date,
           -!!y_signal) %>%
    colnames()

  # Load F-Signals (I)
  cm_names <- read_rds(glue("{path}/Data/ES2/F_Signals/CM_Forecasts_{y_target}.rds")) %>%
    select(-Date) %>%
    colnames()

  # Load F-Signals (II)
  f_names <- read_rds(glue("{path}/Data/ES2/F_Signals/Forecasts_{y_signal}.rds")) %>%
    select(SBAR, UCSV,
           GPR_FAC5, TVD_FAC5, VBDVS_FAC5,
           SSVS_FAC60, VBDVS_FAC60,
           VBDVS_X) %>%
    colnames()

  # Most 15 important signals
  top_signals <- read_rds(glue("{path}/Results/ES2/{setting}_stsc_params_{y_signal}.rds"))$signals %>%
    as_tibble() %>%
    select(where(function(x) any(x > 0))) %>%
    summarise(across(everything(), sum)) %>%
    mutate(row = row_number()) %>%
    pivot_longer(!row) %>%
    arrange(desc(value)) %>%
    select(-row) %>%
    head(n = 15) %>%
    pull(name)

  # Chosen Signals
  df <- read_rds(glue("{path}/Results/ES2/{setting}_stsc_params_{y_signal}.rds"))$signals %>%
    as_tibble(rownames = "Date") %>%
    select(any_of(c("Date", top_signals))) %>%
    mutate(Date = ymd(Date)) %>%
    mutate(t(replicate(nrow(.), seq_len(ncol(.)))) * pick(where(is.numeric))) %>%
    pivot_longer(!Date, names_to = "Predictor", values_to = "value") %>%
    mutate(Indicator = case_when(
      Predictor %in% c(lag_names, simple_names) ~ "P-Signal",
      Predictor %in% c(f_names, cm_names) ~ "F-Signal",
      TRUE ~ "Unknown"
    )) %>%
    mutate(Indicator = factor(Indicator, levels = c("P-Signal",
                                                    "F-Signal",
                                                    "Unkown")))

  # Plot Signals
  plot_sigs <- df %>%
    ggplot() +
    geom_point(aes(Date, value, color = Indicator),
               size = 1.75, alpha = 0.9, shape = 18) +
    labs(title = glue("{plot_name[p]}"),
         y = "Predictive Signals",
         x = "") +
    scale_y_continuous(labels = top_signals,
                       breaks = seq(1, length(top_signals)),
                       limits = c(1, length(top_signals)),
                       expand = y_expand) +
    scale_color_manual(values = c("P-Signal" = "black", "F-Signal" = sky_color),
                       breaks = c("P-Signal", "F-Signal")) +
    plot_aesthetics

  # Save
  write_rds(plot_sigs, glue("{path}/Results/ES2/Plots/{target_names[p]}_vis.rds"))
}

# Define the file names
file_names <- c("GDPCTPI_vis.rds", "PCECTPI_vis.rds", "CPIAUCSL_vis.rds", "CPILFESL_vis.rds")

# Read the files into a list
plots <- lapply(file_names, function(file) {
  read_rds(glue("{path}/Results/ES2/Plots/{file}"))
})

# Create the combined plot
cplot <- cowplot::plot_grid(
  plotlist = plots,
  nrow = 2,
  align = "v"
)

# Save
ggsave("plot_inflation_top15.pdf",
       cplot,
       path = glue::glue("{path}/Results/ES2/Plots/"),
       width = 20,
       height = 15)

# Show Plot
options(repr.plot.width = 20, repr.plot.height = 15)
cplot
######### --------------------------------------------

## 7) LaTex Tables

In [ ]:
######### --------------------------------------------
### Tables Table -- main results ###
target_names <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL")

# Setting
setting <- "inflation_60"

# Load MSE Results
res_mse <- purrr::map(target_names, ~ read_rds(glue("{path}/Results/ES2/{setting}_res_{.x}.rds"))) %>%
  purrr::map(~ .x$mse) %>%
  purrr::map(~ round(.x / .x[, "STSC"], 2)) %>%
  purrr::map(~ as.data.frame(.x)) %>%
  dplyr::bind_rows() %>%
  dplyr::select(-AR2, -STAR2) %>%
  mutate_if(is.numeric, format, digits = 4, nsmall = 2) %>%
  rownames_to_column(" ")

# Load CRPS Results
res_crps <- purrr::map(target_names, ~ read_rds(glue("{path}/Results/ES2/{setting}_res_{.x}.rds"))) %>%
  purrr::map(~ .x$crps) %>%
  purrr::map(~ round(.x / .x[, "STSC"], 2)) %>%
  purrr::map(~ as.data.frame(.x)) %>%
  dplyr::bind_rows() %>%
  dplyr::select(-AR2, -STAR2) %>%
  mutate_if(is.numeric, format, digits = 4, nsmall = 2) %>%
  rownames_to_column(" ")

# Combine Tables
res_mse %>%
  bind_rows(res_crps) %>%
  dplyr::mutate_all(~ stringr::str_replace_all(., "GDPCTPI", "GDP Deflator")) %>%
  dplyr::mutate_all(~ stringr::str_replace_all(., "PCECTPI", "PCE Deflator")) %>%
  dplyr::mutate_all(~ stringr::str_replace_all(., "CPIAUCSL", "Total CPI")) %>%
  dplyr::mutate_all(~ stringr::str_replace_all(., "CPILFESL", "Core CPI")) %>%
  rename("STSC\\textsubscript{ExF}" = STSC_EXF,
         "STSC\\textsubscript{S10}" = STSCS10,
         "STSC\\textsubscript{SFlex}" = STSCSFLEX,
         "PCDMA\\textsubscript{5}" = PC5DMA,
         "PCDMA\\textsubscript{10}" = PC10DMA,
         "PCDMA\\textsubscript{15}" = PC15DMA) %>%
  #replace(is.na(.), "--") %>%
  mutate(across(everything(), ~na_if(.x, "NA")), across(everything(), ~replace_na(.x, "--"))) %>%
  kableExtra::kbl(booktabs = T,
                  caption = "Forecast evaluation for measures of U.S. inflation in terms of MSFE and ACRPS",
                  format = "latex",
                  escape = F) %>%
  kableExtra::kable_styling(latex_options = c("scale_down")) %>%
  kableExtra::pack_rows("MSFE", 1, 4, underline = T) %>%
  kableExtra::pack_rows("ACRPS", 5, 8, underline = T) %>%
  kableExtra::footnote(general = "The table reports MSFE and ACRPS relative to STSC. Values greater (less) than one indicate less (more) accurate forecasts than STSC. The evaluation period spans 1991Q2 to 2021Q4. One star indicates significance at the 10\\\\% level; two stars significance at the 5\\\\% level; and three stars significance at the 1\\\\% level from one-tailed \\\\cite{diebold1995} test statistics compared to forecasts from an AR(2) model.",
                       footnote_as_chunk = TRUE,
                       fixed_small_size = FALSE,
                       threeparttable = TRUE,
                       escape = FALSE,
                       general_title = "") %>%
  kableExtra::row_spec(4, hline_after = T)
######### --------------------------------------------

In [ ]:
######### --------------------------------------------
### LaTex Table -- Robustness
target_names <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL")

# Setting
setting <- "robustness"

# Model Names
robust_names <- c(paste0("$\\kappa=", format(c(0.86, 0.88, 0.90, 0.92, 0.94, 0.96, 0.98, 1.00), digits = 4), "$"),
                  paste0("$\\lambda=", format(c(0.90, 0.925, 0.95, 0.975, 1.00), digits = 4), "$"),
                  paste0("$\\delta=", format(c(0.90, 0.925, 0.95, 0.975, 1.00), digits = 4), "$"),
                  paste0("$\\psi=", c(1, 5, 10, 25, 50, 100, 500, "4,149"), "$"),
                  paste0("$\\gamma=", format(c(0.4, 0.6, 0.8, 0.95, 0.99, 1.00), digits = 2), "$"))

### Load Results: MSE
res_mse <- purrr::map(target_names, ~ read_rds(glue("{path}/Results/ES2/{setting}_res_{.x}.rds"))) %>%
  purrr::map(~ .x$mse) %>%
  purrr::map(~ round(.x / .x[, "STSC"], 2)) %>%
  purrr::map(~ as.data.frame(.x)) %>%
  dplyr::bind_rows() %>%
  dplyr::select(-STAR2, -AR2, -STSC, -STSC_NF) %>%
  `colnames<-`(robust_names) %>%
  t() %>%
  as.data.frame() %>%
  rename("GDP Deflator" = "GDPCTPI",
         "PCE Deflator" = "PCECTPI",
         "Total CPI" = "CPIAUCSL",
         "Core CPI" = "CPILFESL")

# Latex Table
res_mse %>%
  kableExtra::kbl(booktabs = TRUE,
                  caption = "Alternative tuning parameters -- MSFE",
                  format = "latex",
                  escape = FALSE,
                  digits = 2) %>%
  kableExtra::pack_rows("Varying $\\kappa$", 1, 8, underline = T, escape = F) %>%
  kableExtra::pack_rows("Varying $\\lambda$", 9, 13, underline = T, escape = F) %>%
  kableExtra::pack_rows("Varying $\\delta$", 14, 18, underline = T, escape = F) %>%
  kableExtra::pack_rows("Varying $\\psi$", 19, 26, underline = T, escape = F) %>%
  kableExtra::pack_rows("Varying $\\gamma$", 27, 32, underline = T, escape = F) %>%
  kableExtra::footnote(general = "The table summarizes the forecast results in terms of MSFE for alternative values of the tuning parameters.   All results are relative to the unrestricted STSC, with grids for the tuning parameters as described in the main paper. Values less than one indicate better performance than for STSC. In each setup, we varied one of the tuning parameters and fixed its value, while leaving the remaining (grids of) tuning parameters unchanged.",
                       footnote_as_chunk = TRUE,
                       fixed_small_size = FALSE,
                       threeparttable = TRUE,
                       escape = FALSE,
                       general_title = "") %>%
  kableExtra::row_spec(8, hline_after = TRUE) %>%
  kableExtra::row_spec(13, hline_after = TRUE) %>%
  kableExtra::row_spec(18, hline_after = TRUE) %>%
  kableExtra::row_spec(26, hline_after = TRUE) %>%
  kableExtra::row_spec(32, hline_after = TRUE)

### Load Results: CRPS
res_crps <- purrr::map(target_names, ~ read_rds(glue("{path}/Results/ES2/{setting}_res_{.x}.rds"))) %>%
  purrr::map(~ .x$crps) %>%
  purrr::map(~ round(.x / .x[, "STSC"], 2)) %>%
  purrr::map(~ as.data.frame(.x)) %>%
  dplyr::bind_rows() %>%
  dplyr::select(-STAR2, -AR2, -STSC, -STSC_NF) %>%
  `colnames<-`(robust_names) %>%
  t() %>%
  as.data.frame() %>%
  rename("GDP Deflator" = "GDPCTPI",
         "PCE Deflator" = "PCECTPI",
         "Total CPI" = "CPIAUCSL",
         "Core CPI" = "CPILFESL")

# Latex Table
res_crps %>%
  kableExtra::kbl(booktabs = TRUE,
                  caption = "Alternative tuning parameters -- ACRPS",
                  format = "latex",
                  escape = FALSE,
                  digits = 2) %>%
  kableExtra::pack_rows("Varying $\\kappa$", 1, 8, underline = T, escape = F) %>%
  kableExtra::pack_rows("Varying $\\lambda$", 9, 13, underline = T, escape = F) %>%
  kableExtra::pack_rows("Varying $\\delta$", 14, 18, underline = T, escape = F) %>%
  kableExtra::pack_rows("Varying $\\psi$", 19, 26, underline = T, escape = F) %>%
  kableExtra::pack_rows("Varying $\\gamma$", 27, 32, underline = T, escape = F) %>%
  kableExtra::footnote(general = "The table summarizes the forecast results in terms of ACRPS for alternative values of the tuning parameters.   All results are relative to the unrestricted STSC, with grids for the tuning parameters as described in the main paper. Values less than one indicate better performance than for STSC. In each setup, we varied one of the tuning parameters and fixed its value, while leaving the remaining (grids of) tuning parameters unchanged.",
                       footnote_as_chunk = TRUE,
                       fixed_small_size = FALSE,
                       threeparttable = TRUE,
                       escape = FALSE,
                       general_title = "") %>%
  kableExtra::row_spec(8, hline_after = TRUE) %>%
  kableExtra::row_spec(13, hline_after = TRUE) %>%
  kableExtra::row_spec(18, hline_after = TRUE) %>%
  kableExtra::row_spec(26, hline_after = TRUE) %>%
  kableExtra::row_spec(32, hline_after = TRUE)
######### --------------------------------------------

In [1]:
######### --------------------------------------------
### LaTex Table -- Inclusion TVP-VAR
# Target-Variables
target_names <- c("GDPCTPI", "PCECTPI", "CPIAUCSL", "CPILFESL")

# Setting
setting <- "tvp_var"

# CRPS
res_crps <- purrr::map(target_names, ~ read_rds(glue("{path}/Results/ES2/{setting}_res_{.x}.rds"))) %>%
  purrr::map(~ .x$crps) %>%
  purrr::map(~ round(.x / .x[, "STSC"], 2)) %>%
  purrr::map(~ as.data.frame(.x)) %>%
  dplyr::bind_rows() %>%
  dplyr::select(-AR2, -STAR2) %>%
  mutate_if(is.numeric, format, digits = 4, nsmall = 2) %>%
  rownames_to_column(" ") %>%
  dplyr::mutate_all(~ stringr::str_replace_all(., "GDPCTPI", "GDP Deflator")) %>%
  dplyr::mutate_all(~ stringr::str_replace_all(., "PCECTPI", "PCE Deflator")) %>%
  dplyr::mutate_all(~ stringr::str_replace_all(., "CPIAUCSL", "Total CPI")) %>%
  dplyr::mutate_all(~ stringr::str_replace_all(., "CPILFESL", "Core CPI"))

# Latex Table
res_crps %>%
  rename("TVPVAR" = ST_TVP_VAR,
         "STSC\\textsubscript{ExF}" = STSC_EXF) %>%
  kableExtra::kbl(booktabs = T,
                  caption = "Forecast evaluation in terms of ACRPS.",
                  format = "latex",
                  escape = F) %>%
  kableExtra::kable_styling(latex_options = c("scale_down")) %>%
  kableExtra::footnote(general = "The table reports the density predictive accuracy in terms of ACRPS relative to STSC for the OOS evaluation sample from 1991Q4 to 2021Q4. Values less than one indicate better performance than STSC.",
                       footnote_as_chunk = TRUE,
                       fixed_small_size = FALSE,
                       threeparttable = TRUE,
                       escape = FALSE,
                       general_title = "")
######### --------------------------------------------

ERROR: Error in purrr::map(target_names, ~read_rds(glue("{path}/Results/ES2/{setting}_res_{.x}.rds"))) %>% : konnte Funktion "%>%" nicht finden


----